# `ROIC-Preprocessing_ver2.ipynb`

-   ROIC 分析の前処理用 notebook
-   行うこと
    1. 取得したデータをデータベースに落とし込む
    2. ベンチマークのプライスとリターンデータ用意
    3. ファクター計算


In [2]:
%load_ext autoreload
%autoreload 2

import datetime
import gc
import itertools
import os
import sqlite3
import sys
import warnings
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path

import numpy as np
import pandas as pd
import yaml
from dotenv import load_dotenv
from tqdm import tqdm

warnings.simplefilter("ignore")
load_dotenv()

UNIVERSE_CODE = "MSXJPN_AD"
BLOOMBERG_UNIVERSE_TICKER = "MXKO Index"

QUANTS_DIR = Path(os.environ.get("QUANTS_DIR"))  # type: ignore
FACTSET_ROOT_DIR = Path(os.environ.get("FACTSET_ROOT_DIR"))  # type: ignore
FACTSET_FINANCIALS_DIR = Path(os.environ.get("FACTSET_FINANCIALS_DIR"))  # type: ignore
FACTSET_INDEX_CONSTITUENTS_DIR = Path(os.environ.get("FACTSET_INDEX_CONSTITUENTS_DIR"))  # type: ignore
INDEX_DIR = FACTSET_FINANCIALS_DIR / UNIVERSE_CODE
BPM_ROOT_DIR = Path(os.environ.get("BPM_ROOT_DIR"))  # type: ignore
BLOOMBERG_ROOT_DIR = Path(os.environ.get("BLOOMBERG_ROOT_DIR"))  # type: ignore
BLOOMBERG_DATA_DIR = Path(os.environ.get("BLOOMBERG_DATA_DIR"))  # type: ignore

sys.path.insert(0, str(QUANTS_DIR))
import src.calculate_performance_metrics as performance_metrics_utils
import src.database_utils as db_utils
import src.ROIC_make_data_files_ver2 as roic_utils

from src import bloomberg_utils, factset_utils

financials_db_path = INDEX_DIR / "Financials_and_Price.db"
factset_index_db_path = FACTSET_INDEX_CONSTITUENTS_DIR / "Index_Constituents.db"
bloomberg_index_db_path = BLOOMBERG_ROOT_DIR / "Index_Price_and_Returns.db"
bloomberg_valuation_db_path = BLOOMBERG_ROOT_DIR / "Valuation.db"
bpm_db_path = BPM_ROOT_DIR / "Index_Constituents.db"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 0. 現在のデータベースチェック


In [ ]:
tables = sorted(db_utils.get_table_names(financials_db_path))
display(tables)

with sqlite3.connect(financials_db_path) as conn:
    df_tables = pd.read_sql("SELECT * FROM FF_ASSETS", con=conn, parse_dates=["date"])
    display(df_tables)

## 1. BPM と Factset からダウンロードしたデータを sqlite3 に保存

-   インデックス別にテーブルを作成する
-   元データは"Index_Constituents_with_Factset_code-compressed-\*.paruqet" -> 圧縮して送信した
-   BPM から取得した構成比や銘柄 ID などのデータと、Factset でダウンロードした seol, cusip, isin, code_jp にそれぞれ対応する P_SYMBOL および FG_COMPANY_NAME を格納したデータ。


In [ ]:
compressed_files = list(
    FACTSET_INDEX_CONSTITUENTS_DIR.glob(
        "Index_Constituents_with_Factset_code-compressed-*.parquet"
    )
)

dfs = [pd.read_parquet(f) for f in compressed_files]
df = (
    pd.concat(dfs)
    .assign(
        date=lambda x: pd.to_datetime(x["date"]),
        SEDOL=lambda x: x["SEDOL"].astype(str),
    )
    .replace("N/A", np.nan)
)
df[["Holdings", "Weight (%)", "Mkt Value"]] = df[
    ["Holdings", "Weight (%)", "Mkt Value"]
].astype(float)

head_cols = ["Universe", "Universe_code_BPM", "date"]
other_cols = [col for col in df.columns if col not in head_cols]
df = df.reindex(columns=head_cols + other_cols).sort_values(
    ["Universe", "date", "Name"], ignore_index=True
)

for universe_code in df["Universe_code_BPM"].unique():
    df_slice = df.loc[df["Universe_code_BPM"] == universe_code].reset_index(drop=True)
    factset_utils.store_to_database(
        df=df_slice,
        db_path=factset_index_db_path,
        table_name=universe_code,
        unique_cols=["date", "Name", "Asset ID"],
    )

table_names = db_utils.get_table_names(db_path=factset_index_db_path)
display(table_names)

既存の 47072 行との重複をチェックしました。92 行を新たに追加します。
  -> MSSUD: データの書き込みが完了しました。
既存の 319851 行との重複をチェックしました。1182 行を新たに追加します。
  -> MSASD: データの書き込みが完了しました。
既存の 344002 行との重複をチェックしました。1234 行を新たに追加します。
  -> MSACAPFAD: データの書き込みが完了しました。
既存の 247340 行との重複をチェックしました。1054 行を新たに追加します。
  -> MSAPFXJ_AD: データの書き込みが完了しました。
既存の 223596 行との重複をチェックしました。1002 行を新たに追加します。
  -> MSCAFXJAD: データの書き込みが完了しました。
既存の 788150 行との重複をチェックしました。2511 行を新たに追加します。
  -> MSAWIF_AD: データの書き込みが完了しました。
既存の 11887 行との重複をチェックしました。27 行を新たに追加します。
  -> MSCIHKGD: データの書き込みが完了しました。
既存の 23987 行との重複をチェックしました。161 行を新たに追加します。
  -> MSCIINDD: データの書き込みが完了しました。
既存の 7701 行との重複をチェックしました。18 行を新たに追加します。
  -> MSFIDND: データの書き込みが完了しました。
既存の 399723 行との重複をチェックしました。1141 行を新たに追加します。
  -> MSXJPN_AD: データの書き込みが完了しました。
既存の 29542 行との重複をチェックしました。81 行を新たに追加します。
  -> MSFKORD: データの書き込みが完了しました。
既存の 15061 行との重複をチェックしました。27 行を新たに追加します。
  -> MSFMALD: データの書き込みが完了しました。
既存の 5510 行との重複をチェックしました。11 行を新たに追加します。
  -> MSFPHID: データの書き込みが完了しました。
既存の 8856 行との重複をチェックしました。17 行を新たに追加します。
  -> MSCISIN

['MSSUD',
 'MSASD',
 'MSACAPFAD',
 'MSAPFXJ_AD',
 'MSCAFXJAD',
 'MSAWIF_AD',
 'MSCIHKGD',
 'MSCIINDD',
 'MSFIDND',
 'MSXJPN_AD',
 'MSFKORD',
 'MSFMALD',
 'MSFPHID',
 'MSCISIND',
 'MSFTAID',
 'MSFTHAD']

In [ ]:
with sqlite3.connect(factset_index_db_path) as conn:
    df = pd.read_sql(
        f"SELECT * FROM `{UNIVERSE_CODE}`", parse_dates=["date"], con=conn
    ).drop_duplicates()
    df["P_SYMBOL_missing"] = df["P_SYMBOL"].isna()
    display(df)

    g = df.groupby(["date", "P_SYMBOL_missing"])["Weight (%)"].agg(["count", "sum"])
    display(g)

,Universe,Universe_code_BPM,date,Name,Bloomberg Ticker,BloombergID,Asset ID,Asset ID Type,SEDOL,Country,...,P_SYMBOL_CUSIP,ISIN,FG_COMPANY_NAME_ISIN,P_SYMBOL_ISIN,CODE_JP,FG_COMPANY_NAME_CODE_JP,P_SYMBOL_CODE_JP,P_SYMBOL,FG_COMPANY_NAME,P_SYMBOL_missing
0,MSCI KOKUSAI - Daily,MSXJPN_AD,2000-01-31,21ST CENTURY FOX,None,None,AUSBIN2,BARRAID,662075,AUS,...,None,None,None,None,None,None,None,FOXLV-AU,Twenty-First Century Fox Inc. Class A CDI,False
1,MSCI KOKUSAI - Daily,MSXJPN_AD,2000-01-31,21ST CENTURY FOX,None,None,AUSBIN1,BARRAID,688692,AUS,...,None,None,None,None,None,None,None,FOX-AU,Twenty-First Century Fox Inc. Class B CDI,False
2,MSCI KOKUSAI - Daily,MSXJPN_AD,2000-01-31,3I GROUP PLC,None,None,UKIENL1,BARRAID,0888693,GBR,...,TGOPF-US,GB0008886938,スリーアイ・グループ,III-GB,None,None,None,III-GB,スリーアイ・グループ,False
3,MSCI KOKUSAI - Daily,MSXJPN_AD,2000-01-31,3M CO,None,None,USAJ8P1,BARRAID,2595708,USA,...,MMM-US,US6040591058,3Mカンパニー,MMM-US,None,None,None,MMM-US,3Mカンパニー,False
4,MSCI KOKUSAI - Daily,MSXJPN_AD,2000-01-31,ABB LTD,None,None,SWIAAN1,BARRAID,5661190,CHE,...,None,CH0003846620,ABB,ABBN-CH,None,None,None,ABBN-CH,ABB,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400859,MSCI KOKUSAI - Daily,MSXJPN_AD,2025-10-31,ZIMMER BIOMET HOLDINGS INC,None,None,USA4JT1,BARRAID,2783815,USA,...,ZBH-US,US98956P1021,ジンマー・バイオメット・ホールディングス,ZBH-US,None,None,None,ZBH-US,ジンマー・バイオメット・ホールディングス,False
400860,MSCI KOKUSAI - Daily,MSXJPN_AD,2025-10-31,ZOETIS INC,None,None,USBANZ1,BARRAID,B95WG16,USA,...,ZTS-US,US98978V1035,ゾエティス Class A,ZTS-US,None,None,None,ZTS-US,ゾエティス Class A,False
400861,MSCI KOKUSAI - Daily,MSXJPN_AD,2025-10-31,ZOOM COMMUNICATIONS INC,None,None,USBEOV1,BARRAID,BGSP7M9,USA,...,ZM-US,US98980L1017,ズーム・ビデオ・コミュニケーションズ Class A,ZM-US,None,None,None,ZM-US,ズーム・ビデオ・コミュニケーションズ Class A,False
400862,MSCI KOKUSAI - Daily,MSXJPN_AD,2025-10-31,ZSCALER INC,None,None,USBDYI1,BARRAID,BZ00V34,USA,...,ZS-US,US98980G1022,ゼットスケイラー,ZS-US,None,None,None,ZS-US,ゼットスケイラー,False


count         sum
date       P_SYMBOL_missing                   
2000-01-31 False               991   97.959673
           True                 74    2.040327
2000-02-29 False               989   97.900800
           True                 73    2.099194
2000-03-31 False               989   98.129503
...                            ...         ...
2025-07-31 False              1139  100.000012
2025-08-31 False              1140  100.000009
2025-09-30 False              1140   99.999991
2025-10-31 False              1139   99.958187
           True                  2    0.041813

[402 rows x 2 columns]

## 2. Factset からダウンロードしたデータをまとめる

Financials および Price のデータをデータベースに格納する。


In [ ]:
file_list = list(INDEX_DIR.glob("Financials_and_Price-compressed-*.parquet"))
# file_list = [s for s in file_list if not "20241129" in s.name]
dfs = [pd.read_parquet(f) for f in tqdm(file_list, desc="loading parquet files")]
df = (
    pd.concat(dfs)
    .drop_duplicates()
    .sort_values(["variable", "P_SYMBOL", "date"], ignore_index=True)
    .assign(value=lambda x: x["value"].astype(float))
)

for variable in df["variable"].unique():
    df_slice = df.loc[df["variable"] == variable]
    factset_utils.store_to_database(
        df=df_slice,
        db_path=financials_db_path,
        table_name=variable,
        unique_cols=["date", "P_SYMBOL", "variable"],
        verbose=True,
    )

table_names = db_utils.get_table_names(db_path=financials_db_path)
display(table_names)

loading parquet files: 100%|██████████| 9/9 [00:04<00:00,  2.08it/s]


テーブル 'FF_ASSETS' は存在しません。新しいテーブルとして、すべての 791263 行を追加します。
  -> FF_ASSETS: データの書き込みが完了しました。
テーブル 'FF_BPS' は存在しません。新しいテーブルとして、すべての 791263 行を追加します。
  -> FF_BPS: データの書き込みが完了しました。
テーブル 'FF_BPS_TANG' は存在しません。新しいテーブルとして、すべての 791263 行を追加します。
  -> FF_BPS_TANG: データの書き込みが完了しました。
テーブル 'FF_CAPEX' は存在しません。新しいテーブルとして、すべての 791263 行を追加します。
  -> FF_CAPEX: データの書き込みが完了しました。
テーブル 'FF_CASH_ST' は存在しません。新しいテーブルとして、すべての 791263 行を追加します。
  -> FF_CASH_ST: データの書き込みが完了しました。
テーブル 'FF_COGS' は存在しません。新しいテーブルとして、すべての 791263 行を追加します。
  -> FF_COGS: データの書き込みが完了しました。
テーブル 'FF_COM_EQ' は存在しません。新しいテーブルとして、すべての 791263 行を追加します。
  -> FF_COM_EQ: データの書き込みが完了しました。
テーブル 'FF_CURR_RATIO' は存在しません。新しいテーブルとして、すべての 791263 行を追加します。
  -> FF_CURR_RATIO: データの書き込みが完了しました。
テーブル 'FF_DEBT' は存在しません。新しいテーブルとして、すべての 791263 行を追加します。
  -> FF_DEBT: データの書き込みが完了しました。
テーブル 'FF_DEBT_ENTRPR_VAL' は存在しません。新しいテーブルとして、すべての 791263 行を追加します。
  -> FF_DEBT_ENTRPR_VAL: データの書き込みが完了しました。
テーブル 'FF_DEBT_EQ' は存在しません。新しいテーブルとして、すべての 791263 行を追加します。
  -> FF_DEBT_EQ: データの書き込みが

['FF_ASSETS',
 'FF_BPS',
 'FF_BPS_TANG',
 'FF_CAPEX',
 'FF_CASH_ST',
 'FF_COGS',
 'FF_COM_EQ',
 'FF_CURR_RATIO',
 'FF_DEBT',
 'FF_DEBT_ENTRPR_VAL',
 'FF_DEBT_EQ',
 'FF_DEBT_LT',
 'FF_DEBT_ST',
 'FF_DEP_AMORT_EXP',
 'FF_DIV_YLD',
 'FF_DPS',
 'FF_EBITDA_OPER',
 'FF_EBITDA_OPER_MGN',
 'FF_EBIT_OPER',
 'FF_EBIT_OPER_MGN',
 'FF_ENTRPR_VAL_EBITDA_OPER',
 'FF_ENTRPR_VAL_EBIT_OPER',
 'FF_ENTRPR_VAL_SALES',
 'FF_EPS',
 'FF_EPS_DIL',
 'FF_FREE_CF',
 'FF_FREE_PS_CF',
 'FF_GROSS_INC',
 'FF_GROSS_MGN',
 'FF_INC_TAX',
 'FF_INT_EXP_NET',
 'FF_LIABS',
 'FF_LIABS_SHLDRS_EQ',
 'FF_MIN_INT_ACCUM',
 'FF_NET_DEBT',
 'FF_NET_INC',
 'FF_NET_MGN',
 'FF_OPER_CF',
 'FF_OPER_INC',
 'FF_OPER_MGN',
 'FF_OPER_PS_NET_CF',
 'FF_PAY_OUT_RATIO',
 'FF_PBK',
 'FF_PE',
 'FF_PFD_STK',
 'FF_PPE_NET',
 'FF_PSALES',
 'FF_PTX_INC',
 'FF_PTX_MGN',
 'FF_QUICK_RATIO',
 'FF_ROA',
 'FF_ROE',
 'FF_ROIC',
 'FF_ROTC',
 'FF_SALES',
 'FF_SALES_PS',
 'FF_SGA',
 'FF_SHLDRS_EQ',
 'FF_STK_OPT_EXP',
 'FF_STK_PURCH_CF',
 'FF_TAX_RATE',
 'FF_W

### ⚠️1AY/新規データ項目の差分更新がある場合


In [ ]:
def update_value(row, rtol=1e-5, atol=1e-3):
    """既存値をアップデートする関数"""
    existing = row["value_existing"]
    new = row["value_new"]

    # 1. 両方NaN → NaN
    if pd.isna(existing) and pd.isna(new):
        return np.nan

    # 2. 既存がNaN → 新規値
    if pd.isna(existing):
        return new

    # 3. 新規がNaN → 既存値
    if pd.isna(new):
        return existing

    # 4. 両方有効 → 数値比較
    if np.isclose(existing, new, rtol=rtol, atol=atol):
        return existing  # 同じなら既存値
    else:
        return new  # 異なれば新規値で更新


update_file = (
    INDEX_DIR / "Financials_and_Price-compressed-20241129_20251031.parquet"
)  # ファイル名は手動指定する
df_update = pd.read_parquet(update_file)
variable_list = df_update["variable"].sort_values().unique().tolist()
date_list = df_update["date"].sort_values().unique().tolist()
start_date = min(date_list)
end_date = max(date_list)

# データベースの既存テーブル
existing_tables = db_utils.get_table_names(financials_db_path)
# 新しく追加されたデータ項目があるかチェック
added_variables = list(set(variable_list) - set(existing_tables))
if len(added_variables) > 0:
    # 新規データ項目のテーブル作成
    pass

# 1AYで取得したデータを更新するテーブル一覧
update_tables = sorted(list(set(existing_tables) & set(variable_list)))
total_updated = 0
with sqlite3.connect(financials_db_path) as conn:
    for idx, table in enumerate(update_tables, 1):
        # date_listの日付のみデータベースから読み取り、df_updateとの差分を確認
        # 既存のデータが欠損していれば更新データでfillnaする（両方欠損していればそのまま）
        # 既存データと新規データの数値が異なっていれば新規データでupdateする

        print(f"\n[{idx}/{len(update_tables)}] 処理中: {table}")

        query = f"""
            SELECT
                *
            FROM
                {table}
            WHERE
                date >= '{start_date.strftime("%Y-%m-%d")}' AND date <= '{end_date.strftime("%Y-%m-%d")}'
        """
        df_existing = (
            pd.read_sql(query, con=conn, parse_dates=["date"])
            .rename(columns={"value": "value_existing"})
            .reindex(columns=["date", "P_SYMBOL", "variable", "value_existing"])
        )
        df_update_slice = (
            df_update.loc[df_update["variable"] == table]
            .copy()
            .rename(columns={"value": "value_new"})
            .reindex(columns=["date", "P_SYMBOL", "variable", "value_new"])
        )
        df_merged = pd.merge(
            df_update_slice,
            df_existing,
            on=["date", "P_SYMBOL", "variable"],
            how="outer",
        )
        df_merged["value"] = df_merged.apply(update_value, axis=1)

        # 変更検出（ベクトル化版）
        changed_mask = (
            # 既存がNaNで新規に値がある
            (df_merged["value_existing"].isna() & ~df_merged["value_new"].isna())
            |
            # 両方有効で数値が異なる
            (
                ~df_merged["value_existing"].isna()
                & ~df_merged["value_new"].isna()
                & ~np.isclose(
                    df_merged["value_existing"],
                    df_merged["value_new"],
                    rtol=1e-5,
                    atol=1e-3,
                )
            )
        )

        df_to_update = df_merged[changed_mask].reset_index(drop=True)[
            ["date", "P_SYMBOL", "variable", "value"]
        ]

        if len(df_to_update) > 0:
            print(f"  更新対象: {len(df_to_update):,}行")

            # データベース更新
            rows_affected = factset_utils.upsert_financial_data(
                df_to_update, conn, table, method="auto"
            )

            total_updated += rows_affected
        else:
            print("  変更なし")

print(f"\n{'=' * 50}")
print("📊 更新完了")
print(f"{'=' * 50}")
print(f"総更新行数: {total_updated:,}行")


[1/63] 処理中: FF_INT_EXP_NET
  更新対象: 984行
ℹ️  SQLite 3.50.4: upsert方式を使用します
❌ エラー: near "DO": syntax error


OperationalError: near "DO": syntax error

In [ ]:
update_file = (
    INDEX_DIR / "Financials_and_Price-compressed-20241129_20251031.parquet"
)  # ファイル名は手動指定する
df_update = pd.read_parquet(update_file)
variable_list = df_update["variable"].sort_values().unique().tolist()
date_list = df_update["date"].sort_values().unique().tolist()
start_date = min(date_list)
end_date = max(date_list)

# データベースの既存テーブル
existing_tables = db_utils.get_table_names(financials_db_path)
# 新しく追加されたデータ項目があるかチェック
added_variables = list(set(variable_list) - set(existing_tables))
if len(added_variables) > 0:
    # 新規データ項目のテーブル作成
    pass

# 1AYで取得したデータを更新するテーブル一覧
update_tables = list(set(existing_tables) & set(variable_list))
total_updated = 0
with sqlite3.connect(financials_db_path) as conn:
    for idx, table in enumerate(update_tables, 1):
        # date_listの日付のみデータベースから読み取り、df_updateとの差分を確認
        # 既存のデータが欠損していれば更新データでfillnaする（両方欠損していればそのまま）
        # 既存データと新規データの数値が異なっていれば新規データでupdateする

        print(f"\n[{idx}/{len(update_tables)}] 処理中: {table}")

        query = f"""
            SELECT
                *
            FROM
                {table}
            WHERE
                date >= '{start_date.strftime("%Y-%m-%d")}' AND date <= '{end_date.strftime("%Y-%m-%d")}'
        """
        df_existing = (
            pd.read_sql(query, con=conn, parse_dates=["date"])
            .rename(columns={"value": "value_existing"})
            .reindex(columns=["date", "P_SYMBOL", "variable", "value_existing"])
        )
        df_update_slice = (
            df_update.loc[df_update["variable"] == table]
            .copy()
            .rename(columns={"value": "value_new"})
            .reindex(columns=["date", "P_SYMBOL", "variable", "value_new"])
        )
        df_merged = pd.merge(
            df_update_slice,
            df_existing,
            on=["date", "P_SYMBOL", "variable"],
            how="outer",
        )
        df_merged["value"] = df_merged.apply(update_value, axis=1)

        # 変更検出（ベクトル化版）
        changed_mask = (
            # 既存がNaNで新規に値がある
            (df_merged["value_existing"].isna() & ~df_merged["value_new"].isna())
            |
            # 両方有効で数値が異なる
            (
                ~df_merged["value_existing"].isna()
                & ~df_merged["value_new"].isna()
                & ~np.isclose(
                    df_merged["value_existing"],
                    df_merged["value_new"],
                    rtol=1e-5,
                    atol=1e-3,
                )
            )
        )

        df_to_update = df_merged[changed_mask].reset_index(drop=True)[
            ["date", "P_SYMBOL", "variable", "value"]
        ]

        if len(df_to_update) > 0:
            print(f"  更新対象: {len(df_to_update):,}行")

            # データベース更新
            rows_affected = factset_utils.upsert_financial_data(
                df_to_update,
                conn,
                table,
                method="upsert",  # または "upsert"
            )

            total_updated += rows_affected
        else:
            print("  変更なし")

print(f"\n{'=' * 50}")
print("📊 更新完了")
print(f"{'=' * 50}")
print(f"総更新行数: {total_updated:,}行")


[1/63] 処理中: FF_DEBT_ST
  更新対象: 11,175行
⚠️  テーブル 'FF_DEBT_ST' にUNIQUE制約がありません
   制約を追加するには、テーブルを再作成する必要があります
⚠️  UPSERT方式を使用できません。delete_insert方式に切り替えます
  削除: 11175行, 挿入: 11175行
✅ 11175行を処理しました

[2/63] 処理中: FF_TAX_RATE
  更新対象: 10,469行
⚠️  テーブル 'FF_TAX_RATE' にUNIQUE制約がありません
   制約を追加するには、テーブルを再作成する必要があります
⚠️  UPSERT方式を使用できません。delete_insert方式に切り替えます
  削除: 10469行, 挿入: 10469行
✅ 10469行を処理しました

[3/63] 処理中: FF_ROE
  更新対象: 10,836行
⚠️  テーブル 'FF_ROE' にUNIQUE制約がありません
   制約を追加するには、テーブルを再作成する必要があります
⚠️  UPSERT方式を使用できません。delete_insert方式に切り替えます
  削除: 10836行, 挿入: 10836行
✅ 10836行を処理しました

[4/63] 処理中: FF_PPE_NET
  更新対象: 11,147行
⚠️  テーブル 'FF_PPE_NET' にUNIQUE制約がありません
   制約を追加するには、テーブルを再作成する必要があります
⚠️  UPSERT方式を使用できません。delete_insert方式に切り替えます
  削除: 11147行, 挿入: 11147行
✅ 11147行を処理しました

[5/63] 処理中: FF_SGA
  更新対象: 9,573行
⚠️  テーブル 'FF_SGA' にUNIQUE制約がありません
   制約を追加するには、テーブルを再作成する必要があります
⚠️  UPSERT方式を使用できません。delete_insert方式に切り替えます
  削除: 9573行, 挿入: 9573行
✅ 9573行を処理しました

[6/63] 処理中: FF_WKCAP
  更新対象: 11,176行
⚠️  テーブル '

In [ ]:
with sqlite3.connect(financials_db_path) as conn:
    df = pd.read_sql(
        "SELECT * FROM FF_SALES",
        con=conn,
        parse_dates=["date"],
    )
    display(df.dropna(subset=["date"]))

,date,P_SYMBOL,value,variable
0,2005-08-31,0HSW-GB,528.570465,FF_SALES
1,2005-09-30,0HSW-GB,556.819159,FF_SALES
2,2005-10-31,0HSW-GB,556.819159,FF_SALES
3,2005-11-30,0HSW-GB,556.819159,FF_SALES
4,2005-12-30,0HSW-GB,NaN,FF_SALES
...,...,...,...,...
791258,2025-06-30,ZURN-CH,37031.999926,FF_SALES
791259,2025-07-31,ZURN-CH,37031.999926,FF_SALES
791260,2025-08-29,ZURN-CH,37031.999926,FF_SALES
791261,2025-09-30,ZURN-CH,37031.999926,FF_SALES


### データベースチェック


In [ ]:
# データベースの中身チェック
with sqlite3.connect(financials_db_path) as conn:
    df = pd.read_sql(
        "SELECT * FROM FF_ASSETS ORDER BY date", parse_dates=["date"], con=conn
    )
display(df)
display(df.drop_duplicates(subset=["date", "P_SYMBOL"]))
display(df["date"].unique().tolist())

,date,P_SYMBOL,value,variable
0,2005-08-31,0HSW-GB,2448.485005,FF_ASSETS
1,2005-08-31,0II3.XX1-GB,1950.065394,FF_ASSETS
2,2005-08-31,0MDJ-GB,18381.562024,FF_ASSETS
3,2005-08-31,0N1N-GB,2893.729277,FF_ASSETS
4,2005-08-31,0N3I-GB,4176.764098,FF_ASSETS
...,...,...,...,...
791258,2025-10-31,ZBRA-US,8067.000000,FF_ASSETS
791259,2025-10-31,ZM-US,NaN,FF_ASSETS
791260,2025-10-31,ZS-US,NaN,FF_ASSETS
791261,2025-10-31,ZTS-US,15159.000000,FF_ASSETS


,date,P_SYMBOL,value,variable
0,2005-08-31,0HSW-GB,2448.485005,FF_ASSETS
1,2005-08-31,0II3.XX1-GB,1950.065394,FF_ASSETS
2,2005-08-31,0MDJ-GB,18381.562024,FF_ASSETS
3,2005-08-31,0N1N-GB,2893.729277,FF_ASSETS
4,2005-08-31,0N3I-GB,4176.764098,FF_ASSETS
...,...,...,...,...
791258,2025-10-31,ZBRA-US,8067.000000,FF_ASSETS
791259,2025-10-31,ZM-US,NaN,FF_ASSETS
791260,2025-10-31,ZS-US,NaN,FF_ASSETS
791261,2025-10-31,ZTS-US,15159.000000,FF_ASSETS


[Timestamp('2005-08-31 00:00:00'),
 Timestamp('2005-09-30 00:00:00'),
 Timestamp('2005-10-31 00:00:00'),
 Timestamp('2005-11-30 00:00:00'),
 Timestamp('2005-12-30 00:00:00'),
 Timestamp('2006-01-31 00:00:00'),
 Timestamp('2006-02-28 00:00:00'),
 Timestamp('2006-03-31 00:00:00'),
 Timestamp('2006-04-28 00:00:00'),
 Timestamp('2006-05-31 00:00:00'),
 Timestamp('2006-06-30 00:00:00'),
 Timestamp('2006-07-31 00:00:00'),
 Timestamp('2006-08-31 00:00:00'),
 Timestamp('2006-09-29 00:00:00'),
 Timestamp('2006-10-31 00:00:00'),
 Timestamp('2006-11-30 00:00:00'),
 Timestamp('2006-12-29 00:00:00'),
 Timestamp('2007-01-31 00:00:00'),
 Timestamp('2007-02-28 00:00:00'),
 Timestamp('2007-03-30 00:00:00'),
 Timestamp('2007-04-30 00:00:00'),
 Timestamp('2007-05-31 00:00:00'),
 Timestamp('2007-06-29 00:00:00'),
 Timestamp('2007-07-31 00:00:00'),
 Timestamp('2007-08-31 00:00:00'),
 Timestamp('2007-09-28 00:00:00'),
 Timestamp('2007-10-31 00:00:00'),
 Timestamp('2007-11-30 00:00:00'),
 Timestamp('2007-12-

## 3. リターン&ファクター用テーブル作成


### 3-1. リターンのテーブルを作成


In [ ]:
df_price = roic_utils.load_FG_PRICE(db_path=financials_db_path)
df_return = roic_utils.calculate_Return(
    df_price=df_price,
    date_column="date",
    symbol_column="P_SYMBOL",
    price_column="FG_PRICE",
)
print("--- return data ---")
display(df_return.head(3))
print("-" * 20)

# ------------------------------------------------------------------------------------
# データチェック
# 銘柄によってはdateが1カ月ずつ連続でデータがあるとは限らない
# FG_PRICEがない場合にpct_changeを素直に実行するとリターンの期間が他の銘柄とずれる
# そのため、全dateの長さと銘柄ごとのdateの長さを比較する
# ------------------------------------------------------------------------------------

df_check = df_return.reset_index()
symbol_date_counts = df_check.groupby("P_SYMBOL")["date"].nunique()
all_date_len = len(df_check["date"].unique())
not_enough_len_symbols = symbol_date_counts[symbol_date_counts != all_date_len].index
if len(not_enough_len_symbols) > 0:
    print("問題あり")
    display(not_enough_len_symbols)
else:  # 問題なければデータベースに保存
    print("問題なし")
    df_return.reset_index(inplace=True)
    display(df_return.head(5))
    for col in [
        s
        for s in df_return.columns
        if s.startswith("Return") or s.startswith("Forward_Return")
    ]:
        df_slice = (
            df_return[["date", "P_SYMBOL", col]]
            .rename(columns={col: "value"})
            .assign(variable=col)
        )
        df_slice["value"] = df_slice["value"].astype(float)
        df_slice["date"] = pd.to_datetime(df_slice["date"])
        db_utils.delete_table_from_database(db_path=financials_db_path, table_name=col)
        factset_utils.store_to_database(
            df=df_slice, db_path=financials_db_path, table_name=col
        )

--- return data ---


,,FG_PRICE,Return_1M,Forward_Return_1M,Return_1M_annlzd,Forward_Return_1M_annlzd,Return_3M,Forward_Return_3M,Return_3M_annlzd,Forward_Return_3M_annlzd,Return_6M,...,Return_12M_annlzd,Forward_Return_12M_annlzd,Return_3Y,Forward_Return_3Y,Return_3Y_annlzd,Forward_Return_3Y_annlzd,Return_5Y,Forward_Return_5Y,Return_5Y_annlzd,Forward_Return_5Y_annlzd
P_SYMBOL,date,,,,,,,,,,,,,,,,,,,,,
0HSW-GB,2005-08-31,3.23,NaN,0.13,NaN,1.53,NaN,0.34,NaN,1.37,NaN,...,NaN,0.55,NaN,0.85,NaN,0.28,NaN,0.85,NaN,0.17
0II3.XX1-GB,2005-08-31,4.17,NaN,0.01,NaN,0.18,NaN,0.06,NaN,0.24,NaN,...,NaN,-0.12,NaN,0.12,NaN,0.04,NaN,0.12,NaN,0.02
0MDJ-GB,2005-08-31,6.09,NaN,0.05,NaN,0.55,NaN,0.02,NaN,0.06,NaN,...,NaN,0.02,NaN,0.04,NaN,0.01,NaN,0.42,NaN,0.08


--------------------
問題なし


,P_SYMBOL,date,FG_PRICE,Return_1M,Forward_Return_1M,Return_1M_annlzd,Forward_Return_1M_annlzd,Return_3M,Forward_Return_3M,Return_3M_annlzd,...,Return_12M_annlzd,Forward_Return_12M_annlzd,Return_3Y,Forward_Return_3Y,Return_3Y_annlzd,Forward_Return_3Y_annlzd,Return_5Y,Forward_Return_5Y,Return_5Y_annlzd,Forward_Return_5Y_annlzd
0,0HSW-GB,2005-08-31,3.23,NaN,0.13,NaN,1.53,NaN,0.34,NaN,...,NaN,0.55,NaN,0.85,NaN,0.28,NaN,0.85,NaN,0.17
1,0II3.XX1-GB,2005-08-31,4.17,NaN,0.01,NaN,0.18,NaN,0.06,NaN,...,NaN,-0.12,NaN,0.12,NaN,0.04,NaN,0.12,NaN,0.02
2,0MDJ-GB,2005-08-31,6.09,NaN,0.05,NaN,0.55,NaN,0.02,NaN,...,NaN,0.02,NaN,0.04,NaN,0.01,NaN,0.42,NaN,0.08
3,0N1N-GB,2005-08-31,5.66,NaN,0.04,NaN,0.47,NaN,0.02,NaN,...,NaN,0.05,NaN,0.40,NaN,0.13,NaN,0.37,NaN,0.07
4,0N3I-GB,2005-08-31,2.81,NaN,0.03,NaN,0.34,NaN,0.03,NaN,...,NaN,0.01,NaN,-0.47,NaN,-0.16,NaN,0.15,NaN,0.03


テーブル 'Return_1M' は存在しません。新しいテーブルとして、すべての 794853 行を追加します。
  -> Return_1M: データの書き込みが完了しました。
テーブル 'Forward_Return_1M' は存在しません。新しいテーブルとして、すべての 794853 行を追加します。
  -> Forward_Return_1M: データの書き込みが完了しました。
テーブル 'Return_1M_annlzd' は存在しません。新しいテーブルとして、すべての 794853 行を追加します。
  -> Return_1M_annlzd: データの書き込みが完了しました。
テーブル 'Forward_Return_1M_annlzd' は存在しません。新しいテーブルとして、すべての 794853 行を追加します。
  -> Forward_Return_1M_annlzd: データの書き込みが完了しました。
テーブル 'Return_3M' は存在しません。新しいテーブルとして、すべての 794853 行を追加します。
  -> Return_3M: データの書き込みが完了しました。
テーブル 'Forward_Return_3M' は存在しません。新しいテーブルとして、すべての 794853 行を追加します。
  -> Forward_Return_3M: データの書き込みが完了しました。
テーブル 'Return_3M_annlzd' は存在しません。新しいテーブルとして、すべての 794853 行を追加します。
  -> Return_3M_annlzd: データの書き込みが完了しました。
テーブル 'Forward_Return_3M_annlzd' は存在しません。新しいテーブルとして、すべての 794853 行を追加します。
  -> Forward_Return_3M_annlzd: データの書き込みが完了しました。
テーブル 'Return_6M' は存在しません。新しいテーブルとして、すべての 794853 行を追加します。
  -> Return_6M: データの書き込みが完了しました。
テーブル 'Forward_Return_6M' は存在しません。新しいテーブルとして、すべての 794853 行を追加します。
  -> 

### 3-2. インデックスの価格とリターンデータを取得 via Blpapi

⚠️ 注意 ⚠️ Bloomberg Terminal を起動している必要あり。


In [ ]:
# データベース確認
with sqlite3.connect(bloomberg_index_db_path) as conn:
    df = pd.read_sql("SELECT * FROM PX_LAST", con=conn, parse_dates=["Date"])
display(df)
print(
    f"Date: {df['Date'].min().strftime('%Y-%m-%d')} 〜 {df['Date'].max().strftime('%Y-%m-%d')} ({len(df['Date'].unique()):,}日)"
)
print(f"Ticker: {df['Ticker'].nunique():,}銘柄\n\t{df['Ticker'].unique().tolist()}")

DatabaseError: Execution failed on sql 'SELECT * FROM PX_LAST': no such table: PX_LAST

In [ ]:
# yamlで設定した銘柄リストの読み込み（Bloomberg Ticker）
BLOOMBERG_TICKER_YAML = BLOOMBERG_ROOT_DIR / "ticker-description.yaml"
EQUITY_TYPES = {"equity_index", "equity_sector_index", "equity_industry_index"}

with open(BLOOMBERG_TICKER_YAML, "r", encoding="utf-8") as f:
    ticker_descriptions = yaml.safe_load(f)

tickers_to_download = [
    ticker["bloomberg_ticker"]
    for ticker in ticker_descriptions
    if ticker.get("type") in EQUITY_TYPES
]

# -----------------------------------------------------
# Bloombergから価格データをダウンロードしてデータベースを更新
# -----------------------------------------------------


blp = bloomberg_utils.BlpapiCustom()
# 新規ティッカーがある場合

# df = blp.get_historical_data(
#     securities=tickers_to_download,
#     fields=["PX_LAST"],
#     start_date="20000101",
#     end_date=datetime.datetime.today().strftime("%Y%m%d"),
# )
# df = pd.melt(
#     df.reset_index(), id_vars=["Date"], var_name="Ticker", value_name="value"
# ).assign(variable="PX_LAST")
# display(df)
# blp.store_to_database(
#     df=df,
#     db_path=bloomberg_index_db_path,
#     table_name="PX_LAST",
#     primary_keys=["Date", "Ticker", "variable"],
#     verbose=True,
# )

# 　既存データの更新

rows_updated = blp.update_historical_data(
    db_path=bloomberg_index_db_path,
    table_name="PX_LAST",
    tickers=tickers_to_download,
    id_type="ticker",
    field="PX_LAST",
    default_start_date=datetime.datetime(2000, 1, 1),
    verbose=True,
)
print(f"\n{'=' * 60}")
print(f"処理完了: {rows_updated:,}行を処理しました")
print(f"{'=' * 60}")

🆕 初期データ取得モード
   取得期間: 2000-01-01 ~ 2025-12-08
   対象銘柄: 70銘柄
Bloombergセッションを開始しています...
セッション開始成功。
✅ サービスオープン完了。リクエスト作成中...
📡 リクエストを送信します [DAILY]
   期間: 2000-01-01 - 2025-12-08
❌ MXWDJ0HC Index (TICKER) でエラー: Unknown/Invalid securityInvalid Security [nid:53728]
❌ MXWDJ0IN Index (TICKER) でエラー: Unknown/Invalid securityInvalid Security [nid:53728]
❌ MXWDJ0IT Index (TICKER) でエラー: Unknown/Invalid securityInvalid Security [nid:53728]
❌ MXWDJ0MT Index (TICKER) でエラー: Unknown/Invalid securityInvalid Security [nid:53728]
❌ MXWDJ0RE Index (TICKER) でエラー: Unknown/Invalid securityInvalid Security [nid:53728]
❌ MXWDJ0UT Index (TICKER) でエラー: Unknown/Invalid securityInvalid Security [nid:53728]
❌ MXWDJ0CS Index (TICKER) でエラー: Unknown/Invalid securityInvalid Security [nid:53822]
❌ MXWDJ0CD Index (TICKER) でエラー: Unknown/Invalid securityInvalid Security [nid:53822]
❌ MXWDJ0ST Index (TICKER) でエラー: Unknown/Invalid securityInvalid Security [nid:53822]
❌ MXWDJ0EN Index (TICKER) でエラー: Unknown/Invalid securityInva

In [ ]:
# 既存のFG_PRICEのテーブルからリターン計算すべき日付を取得
df_index_price_filtered = (
    db_utils.get_rows_by_unique_values(
        source_db_path=financials_db_path,
        target_db_path=bloomberg_index_db_path,
        source_table="FG_PRICE",
        target_table="PX_LAST",
        source_column="date",
        target_column="Date",
    )
    .query(f"Ticker == '{BLOOMBERG_UNIVERSE_TICKER}'")
    .reset_index(drop=True)
    .assign(Date=lambda x: pd.to_datetime(x["Date"]))
)


# インデックスについて同様にリターンを計算してデータベースに保存
df_index_return = roic_utils.calculate_Return(
    df_price=df_index_price_filtered,
    date_column="Date",
    symbol_column="Ticker",
    price_column="value",
)
print("--- return data ---")
display(df_index_return.tail(3))
print("-" * 20)

# ------------------------------------------------------------------------------------
# データチェック
# 銘柄によってはdateが1カ月ずつ連続でデータがあるとは限らない
# 価格データがない場合にpct_changeを素直に実行するとリターンの期間が他の銘柄とずれる
# そのため、全dateの長さと銘柄ごとのdateの長さを比較する
# ------------------------------------------------------------------------------------

df_check = df_index_return.reset_index()
symbol_date_counts = df_check.groupby("Ticker")["Date"].nunique()
all_date_len = len(df_check["Date"].unique())
not_enough_len_symbols = symbol_date_counts[symbol_date_counts != all_date_len].index
if len(not_enough_len_symbols) > 0:
    print("問題あり")
    display(not_enough_len_symbols)
else:  # 問題なければデータベースに保存
    print("問題なし")
    df_index_return.reset_index(inplace=True)
    display(df_index_return.tail(5))
    for col in [
        s
        for s in df_index_return.columns
        if s.startswith("Return") or s.startswith("Forward_Return")
    ]:
        df_slice = (
            df_index_return[["Date", "Ticker", col]]
            .rename(columns={col: "value"})
            .assign(variable=col)
        )
        df_slice["value"] = df_slice["value"].astype(float)
        df_slice["Date"] = pd.to_datetime(df_slice["Date"])
        db_utils.delete_table_from_database(
            db_path=bloomberg_index_db_path, table_name=col
        )
        blp.store_to_database(
            df=df_slice,
            db_path=bloomberg_index_db_path,
            table_name=col,
            primary_keys=["Date", "Ticker", "variable"],
        )

--- return data ---


value variable  Return_1M  Forward_Return_1M  \
Ticker     Date                                                         
MXKO Index 2025-08-29  4339.70  PX_LAST   0.022468           0.031714   
           2025-09-30  4477.33  PX_LAST   0.031714           0.018609   
           2025-10-31  4560.65  PX_LAST   0.018609                NaN   

                       Return_1M_annlzd  Forward_Return_1M_annlzd  Return_3M  \
Ticker     Date                                                                
MXKO Index 2025-08-29          0.269611                  0.380570   0.081942   
           2025-09-30          0.380570                  0.223312   0.069486   
           2025-10-31          0.223312                       NaN   0.074525   

                       Forward_Return_3M  Return_3M_annlzd  \
Ticker     Date                                              
MXKO Index 2025-08-29                NaN          0.327766   
           2025-09-30                NaN          0.277946   
           2025-10-31                NaN          0.298101   

                       Forward_Return_3M_annlzd  ...  Return_12M_annlzd  \
Ticker     Date                                  ...                      
MXKO Index 2025-08-29                       NaN  ...           0.143251   
           2025-09-30                       NaN  ...           0.157818   
           2025-10-31                       NaN  ...           0.202551   

                       Forward_Return_12M_annlzd  Return_3Y  \
Ticker     Date                                               
MXKO Index 2025-08-29                        NaN   0.598328   
           2025-09-30                        NaN   0.819180   
           2025-10-31                        NaN   0.725760   

                       Forward_Return_3Y  Return_3Y_annlzd  \
Ticker     Date                                              
MXKO Index 2025-08-29                NaN          0.199443   
           2025-09-30                NaN          0.273060   
           2025-10-31                NaN          0.241920   

                       Forward_Return_3Y_annlzd  Return_5Y  Forward_Return_5Y  \
Ticker     Date                                                                 
MXKO Index 2025-08-29                       NaN   0.726859                NaN   
           2025-09-30                       NaN   0.854051                NaN   
           2025-10-31                       NaN   0.952408                NaN   

                       Return_5Y_annlzd  Forward_Return_5Y_annlzd  
Ticker     Date                                                    
MXKO Index 2025-08-29          0.145372                       NaN  
           2025-09-30          0.170810                       NaN  
           2025-10-31          0.190482                       NaN  

[3 rows x 26 columns]

--------------------
問題なし


,Ticker,Date,value,variable,Return_1M,Forward_Return_1M,Return_1M_annlzd,Forward_Return_1M_annlzd,Return_3M,Forward_Return_3M,...,Return_12M_annlzd,Forward_Return_12M_annlzd,Return_3Y,Forward_Return_3Y,Return_3Y_annlzd,Forward_Return_3Y_annlzd,Return_5Y,Forward_Return_5Y,Return_5Y_annlzd,Forward_Return_5Y_annlzd
238,MXKO Index,2025-06-30,4186.43,PX_LAST,0.043729,0.013833,0.524753,0.165993,0.109511,0.069486,...,0.148449,NaN,0.591242,NaN,0.197081,NaN,0.865951,NaN,0.173190,NaN
239,MXKO Index,2025-07-31,4244.34,PX_LAST,0.013833,0.022468,0.165993,0.269611,0.119433,0.074525,...,0.147621,NaN,0.493686,NaN,0.164562,NaN,0.797764,NaN,0.159553,NaN
240,MXKO Index,2025-08-29,4339.70,PX_LAST,0.022468,0.031714,0.269611,0.380570,0.081942,NaN,...,0.143251,NaN,0.598328,NaN,0.199443,NaN,0.726859,NaN,0.145372,NaN
241,MXKO Index,2025-09-30,4477.33,PX_LAST,0.031714,0.018609,0.380570,0.223312,0.069486,NaN,...,0.157818,NaN,0.819180,NaN,0.273060,NaN,0.854051,NaN,0.170810,NaN
242,MXKO Index,2025-10-31,4560.65,PX_LAST,0.018609,NaN,0.223312,NaN,0.074525,NaN,...,0.202551,NaN,0.725760,NaN,0.241920,NaN,0.952408,NaN,0.190482,NaN


✅ 保存完了。テーブル 'Return_1M' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Forward_Return_1M' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Return_1M_annlzd' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Forward_Return_1M_annlzd' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Return_3M' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Forward_Return_3M' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Return_3M_annlzd' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Forward_Return_3M_annlzd' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Return_6M' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Forward_Return_6M' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Return_6M_annlzd' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Forward_Return_6M_annlzd' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Return_12M' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Forward_Return_12M' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Return_12M_annlzd' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Forward_Return_12M_annlzd' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Return_3Y' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Forward_Return_3Y' に 243 行を処理し

### 3-3. Active Return 計算

テーブル作成し、データベースに保存


In [ ]:
return_cols = [
    "Return_1M",
    "Return_1M_annlzd",
    "Forward_Return_1M",
    "Forward_Return_1M_annlzd",
    "Return_3M",
    "Return_3M_annlzd",
    "Forward_Return_3M",
    "Forward_Return_3M_annlzd",
    "Return_6M",
    "Return_6M_annlzd",
    "Forward_Return_6M",
    "Forward_Return_6M_annlzd",
    "Return_12M",
    "Return_12M_annlzd",
    "Forward_Return_12M",
    "Forward_Return_12M_annlzd",
    "Return_3Y",
    "Return_3Y_annlzd",
    "Forward_Return_3Y",
    "Forward_Return_3Y_annlzd",
    "Return_5Y",
    "Return_5Y_annlzd",
    "Forward_Return_5Y",
    "Forward_Return_5Y_annlzd",
]

# ------------------------------------
# インデックスのリターン
# ------------------------------------
union_queries_index = []
for table in return_cols:
    union_queries_index.append(
        f"SELECT Date, Ticker, value, variable FROM '{table}' WHERE Ticker = '{BLOOMBERG_UNIVERSE_TICKER}'"
    )
union_query_index = " UNION ALL ".join(union_queries_index)

# データ取得
with sqlite3.connect(bloomberg_index_db_path) as conn:
    df_return_index = pd.read_sql(
        union_query_index, con=conn, parse_dates=["Date"]
    ).rename(columns={"Date": "date", "Ticker": "symbol"})

df_return_index = df_return_index.drop_duplicates(ignore_index=True)
print(f"✅ インデックスデータ: {len(df_return_index):,}件")


# ------------------------------------
# 個別銘柄のリターン
# ------------------------------------
union_queries_security = []
for table in return_cols:
    union_queries_security.append(
        f"SELECT date, P_SYMBOL, value, variable FROM '{table}'"
    )

union_query_security = " UNION ALL ".join(union_queries_security)

# データ取得
with sqlite3.connect(financials_db_path) as conn:
    df_return_security = pd.read_sql(
        union_query_security, con=conn, parse_dates=["date"]
    ).rename(columns={"P_SYMBOL": "symbol"})

df_return_security = df_return_security.drop_duplicates(ignore_index=True)

print(f"✅ 個別銘柄データ: {len(df_return_security):,}件")

# ------------------------------------
# concatenate returns
# ------------------------------------
df_returns = pd.concat([df_return_index, df_return_security], ignore_index=True)
display(df_returns.tail(3))

# ------------------------------------
# アクティブリターン計算
# ------------------------------------
df_active_returns = performance_metrics_utils.calculate_active_returns_vectorized(
    df_returns=df_returns,
    return_cols=return_cols,
    benchmark_ticker=BLOOMBERG_UNIVERSE_TICKER,
    verbose=False,
)
display(df_active_returns.tail(3))

# ------------------------------------
# Active return: データベース保存
# ------------------------------------
# 推奨方法：直列書き込み版
results = factset_utils.insert_active_returns_optimized_sqlite(
    df_active_returns=df_active_returns,
    return_cols=return_cols,
    db_path=financials_db_path,
    benchmark_ticker=BLOOMBERG_UNIVERSE_TICKER,
    batch_size=10000,
    verbose=True,
)

✅ インデックスデータ: 5,832件
✅ 個別銘柄データ: 19,076,472件


,date,symbol,value,variable
19082301,2025-10-31,ZS-US,NaN,Forward_Return_5Y_annlzd
19082302,2025-10-31,ZTS-US,NaN,Forward_Return_5Y_annlzd
19082303,2025-10-31,ZURN-CH,NaN,Forward_Return_5Y_annlzd


,date,symbol,value,variable
19076469,2025-08-29,ZURN-CH,NaN,Forward_Active_Return_5Y_annlzd
19076470,2025-09-30,ZURN-CH,NaN,Forward_Active_Return_5Y_annlzd
19076471,2025-10-31,ZURN-CH,NaN,Forward_Active_Return_5Y_annlzd


⚡ アクティブリターン最適化バッチ保存（SQLite機能活用版）
   処理列数: 24列
   データ行数: 19,076,472行
   バッチサイズ (executemany): 10,000行
⏳ データ前処理中...
✅ 前処理完了 (69.37秒)
   処理対象: 24テーブル


💾 保存中:   4%|▍         | 1/24 [00:09<03:36,  9.43s/it]

...

✅ Active_Return_5Y_annlzd: 794,853件（挿入試行）


💾 保存中:  96%|█████████▌| 23/24 [03:57<00:10, 10.44s/it]

✅ Forward_Active_Return_5Y: 794,853件（挿入試行）


💾 保存中: 100%|██████████| 24/24 [04:10<00:00, 10.42s/it]

✅ Forward_Active_Return_5Y_annlzd: 794,853件（挿入試行）
📊 バッチ保存完了統計
   成功: 24/24テーブル
   失敗: 0テーブル
   総試行件数: 19,076,472件
   前処理時間: 69.37秒
   保存時間: 250.06秒
   合計時間: 319.43秒
   スループット (試行件数ベース): 59,720件/秒
   成功率: 100.0%


### 🧪PE 取得テスト from Bloomberg（⚠️ データ取得は難しそう）

-   Forward PE(BEST_PE_RATIO), Trailing PE(PE_RATIO), と Forward EPS(BEST_EPS), Trailing EPS(TRAIL_12M_EPS_BEF_XO_ITEM)を取得

-   (- Forward PE と Trailing PE の差分のカラムを追加
-   PEG ratio = BEST_EPS / BEST_PE_RATIO のカラムを追加
    )


In [ ]:
fields = ["BEST_PE_RATIO", "BEST_EPS", "PE_RATIO", "TRAIL_12M_EPS_BEF_XO_ITEM"]
# --------------------------------------------------------
# sedolとdateのリストを取得
# --------------------------------------------------------
with sqlite3.connect(factset_index_db_path) as conn:
    sedol_list = pd.read_sql(f"SELECT DISTINCT `SEDOL` FROM {UNIVERSE_CODE}", con=conn)[
        "SEDOL"
    ].tolist()
    sedol_list = [s + " Equity" for s in sedol_list]
    date_list = pd.read_sql(
        f"SELECT DISTINCT `date` FROM {UNIVERSE_CODE} ORDER BY `date`",
        con=conn,
        parse_dates=["date"],
    )["date"].tolist()
print("SEDOLと日付のリスト取得完了")

blp = bloomberg_utils.BlpapiCustom()
for field in fields:
    df = (
        blp.get_historical_data_with_overrides(
            securities=sedol_list,
            id_type="sedol",
            fields=[field],
            start_date=min(date_list).strftime("%Y%m%d"),
            end_date=max(date_list).strftime("%Y%m%d"),
            periodicity="MONTHLY",
            # overrides={"BEST_FPERIOD_OVERRIDE": "1FQ"},   # デフォルトで1FQ
            verbose=True,
        )
        .sort_values(["Date", "Identifier"], ignore_index=True)
        .drop(columns=["ID_Type"])
        .assign(Date=lambda x: pd.to_datetime(x["Date"]))
    )

    df = (
        pd.melt(
            df,
            id_vars=["Date", "Identifier"],
            value_vars=[field],
            var_name="variable",
        )
        .rename(columns={"Identifier": "SEDOL"})
        .assign(SEDOL=lambda x: x["SEDOL"].str.replace(" Equity", ""))
    )

    # blp.store_to_database(
    #     df=df,
    #     db_path=bloomberg_valuation_db_path,
    #     table_name=field,
    #     primary_keys=["Date", "SEDOL", "variable"],
    # )

SEDOLと日付のリスト取得完了
Bloombergセッションを開始しています...
セッション開始成功。
✅ サービスオープン完了。リクエスト作成中...
📡 リクエストを送信します[SEDOL] [MONTHLY]
   期間: 2000-01-31 - 2025-10-31

✅ データ取得完了。接続を終了しました。

📊 取得データ:
   行数: 21行
   日付範囲: 2023-11-30 00:00:00 ~ 2025-07-31 00:00:00
   ユニーク日数: 21日
   識別子数: 1
   識別子タイプ: SEDOL
   周期: MONTHLY
Bloombergセッションを開始しています...
セッション開始成功。
✅ サービスオープン完了。リクエスト作成中...
📡 リクエストを送信します[SEDOL] [MONTHLY]
   期間: 2000-01-31 - 2025-10-31

✅ データ取得完了。接続を終了しました。

📊 取得データ:
   行数: 13行
   日付範囲: 2023-11-30 00:00:00 ~ 2025-09-30 00:00:00
   ユニーク日数: 13日
   識別子数: 1
   識別子タイプ: SEDOL
   周期: MONTHLY
Bloombergセッションを開始しています...
セッション開始成功。
✅ サービスオープン完了。リクエスト作成中...
📡 リクエストを送信します[SEDOL] [MONTHLY]
   期間: 2000-01-31 - 2025-10-31

✅ データ取得完了。接続を終了しました。

📊 取得データ:
   行数: 305行
   日付範囲: 2000-01-31 00:00:00 ~ 2025-10-31 00:00:00
   ユニーク日数: 305日
   識別子数: 1
   識別子タイプ: SEDOL
   周期: MONTHLY
Bloombergセッションを開始しています...
セッション開始成功。
✅ サービスオープン完了。リクエスト作成中...
📡 リクエストを送信します[SEDOL] [MONTHLY]
   期間: 2000-01-31 - 2025-10-31

✅ データ取得完了。接続を終了しました。

📊 取得デー

✅ データベースチェック


In [ ]:
with sqlite3.connect(bloomberg_valuation_db_path) as conn:
    df = pd.read_sql("SELECT * FROM BEST_PE_RATIO", con=conn, parse_dates=["Date"])
    display(df["Date"].sort_values().unique())
    display(df.sort_values("Date", ignore_index=True).drop_duplicates())

<DatetimeArray>
['2000-01-31 00:00:00', '2000-02-29 00:00:00', '2000-03-31 00:00:00',
 '2000-04-28 00:00:00', '2000-05-31 00:00:00', '2000-06-30 00:00:00',
 '2000-07-31 00:00:00', '2000-08-31 00:00:00', '2000-09-29 00:00:00',
 '2000-10-31 00:00:00',
 ...
 '2025-04-30 00:00:00', '2025-05-29 00:00:00', '2025-05-30 00:00:00',
 '2025-06-30 00:00:00', '2025-07-31 00:00:00', '2025-08-29 00:00:00',
 '2025-08-31 00:00:00', '2025-09-30 00:00:00', '2025-10-30 00:00:00',
 '2025-10-31 00:00:00']
Length: 334, dtype: datetime64[ns]

,Date,SEDOL,variable,value
0,2000-01-31,2206301,BEST_PE_RATIO,11.744
1,2000-01-31,2213981,BEST_PE_RATIO,10.710
2,2000-02-29,2206301,BEST_PE_RATIO,13.396
3,2000-02-29,2213981,BEST_PE_RATIO,10.454
4,2000-03-31,2206301,BEST_PE_RATIO,14.650
...,...,...,...,...
61076,2025-10-31,2852533,BEST_PE_RATIO,26.723
61077,2025-10-31,2849472,BEST_PE_RATIO,13.087
61078,2025-10-31,2842255,BEST_PE_RATIO,33.487
61079,2025-10-31,2838555,BEST_PE_RATIO,22.003


✅ 欠損割合チェック


In [ ]:
df_weight = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
)

# bloomberg per
with sqlite3.connect(bloomberg_valuation_db_path) as conn:
    df_forward_pe = pd.read_sql(
        "SELECT * FROM BEST_PE_RATIO", parse_dates=["Date"], con=conn
    ).rename(columns={"value": "BEST_PE_RATIO", "Date": "date"})
    df_actual_pe = pd.read_sql(
        "SELECT * FROM PE_RATIO", parse_dates=["Date"], con=conn
    ).rename(columns={"value": "PE_RATIO", "Date": "date"})
    df_actual_pe.drop(columns=["variable"], inplace=True)
    df_pe = pd.merge(
        df_forward_pe, df_actual_pe, on=["date", "SEDOL"], how="outer"
    ).assign(date=lambda x: x["date"] + pd.offsets.MonthEnd(0))

df_merged = pd.merge(df_weight, df_pe, on=["date", "SEDOL"], how="outer").dropna(
    subset=["Weight (%)", "BEST_PE_RATIO", "PE_RATIO"], how="any", ignore_index=True
)

display(df_merged.tail(5))

,date,P_SYMBOL,SEDOL,FG_COMPANY_NAME,GICS Sector,GICS Industry Group,Weight (%),variable,BEST_PE_RATIO,PE_RATIO
39444,2025-10-31,SBAC-US,BZ6TS23,SBAコミュニケーションズ Class A,Real Estate,Equity Real Estate Investment Trusts (REITs),0.026502,BEST_PE_RATIO,22.317,20.7390
39445,2025-10-31,CHTR-US,BZ6VT82,チャーター・コミュニケーションズ Class A,Communication Services,Media & Entertainment,0.026749,BEST_PE_RATIO,5.755,5.8559
39446,2025-10-31,WTC-AU,BZ8GX83,ワイズテック・グローバル,Information Technology,Software & Services,0.011702,BEST_PE_RATIO,57.203,74.8462
39447,2025-10-31,VST-US,BZ8VJQ8,ビストラ,Utilities,Utilities,0.082303,BEST_PE_RATIO,21.327,39.5135
39448,2025-10-31,BCP-PT,BZCNN35,Banco Comercial Portugues S.A.,Financials,Banks,0.009457,BEST_PE_RATIO,10.628,11.8584


In [ ]:
# 欠損
g = pd.DataFrame(df_merged.groupby(["date"])["Weight (%)"].agg("sum")).reset_index()
display(g.query("date>='2023-01-01'"))

,date,Weight (%)
271,2023-01-31,54.865295
272,2023-02-28,58.768044
273,2023-03-31,61.187943
274,2023-04-30,61.089016
275,2023-05-31,62.986537
276,2023-06-30,63.706548
277,2023-07-31,64.796422
278,2023-08-31,66.780949
279,2023-09-30,67.820945
280,2023-10-31,67.305035


#### Bloomberg valuation データについてメモ

-   直近 1,2 年分程度しか forward+actual pe は取れない


## 4. Factor 計算


### 4-1. 成長率計算（Factset）


#### 1. QoQ, YoY, 3Yr CAGR, 5Yr CAGR の値を計算し、データベースに保存する。


In [ ]:
factor_list = [
    "FF_SALES",
    "FF_EBITDA_OPER",
    "FF_EBIT_OPER",
    "FF_EPS",
    "FF_OPER_CF",
    "FF_ASSETS",
    "FF_COM_EQ",
    "FF_DEBT",
    "FF_DEBT_LT",
    "FF_DEBT_ST",
    "FF_OPER_INC",
    "FF_CAPEX",
    "FF_FREE_CF",
    "FF_EPS_DIL",
]
period_list = ["QoQ", "YoY", "CAGR_3Y", "CAGR_5Y"]

In [ ]:
query = [f"SELECT * FROM `{table}`" for table in factor_list]
query = " UNION ALL ".join(query)

# ------------------------------------------------------
# load from database
# ------------------------------------------------------
with sqlite3.connect(financials_db_path) as conn:
    df_all = (
        pd.read_sql(query, con=conn, parse_dates=["date"])
        .sort_values("date", ignore_index=True)
        .assign(
            variable=lambda x: x["variable"].astype("category"),
            P_SYMBOL=lambda x: x["P_SYMBOL"].astype("category"),
        )
    ).sort_values(["variable", "P_SYMBOL", "date"], ignore_index=True)

display(df_all)
# groupby("variable")を使うことで、df_all全体を何度も走査(loc)するコストをゼロにする
# observed=True はcategory型を使う場合の高速化オプション
grouped = df_all.groupby("variable", observed=True)

# ------------------------------------------------------
# calculate growth and store to database
# ------------------------------------------------------
total_steps = len(factor_list)
for factor_name, df_factor in tqdm(grouped, total=total_steps, desc="Factors"):
    # df_factorはView(参照)の可能性があるため、計算用にコピーを作成
    # ここでメモリを食うが、factor単位なので全体コピーよりは軽い
    # かつ、ループの最後で解放される
    df_base = df_factor.copy()

    for growth in period_list:
        new_variable_name = f"{factor_name}_{growth}"
        df_result = roic_utils.calculate_growth(
            df=df_base, data_name=str(factor_name), growth_type=growth
        )

        # store to database
        db_utils.delete_table_from_database(
            db_path=financials_db_path, table_name=new_variable_name
        )
        factset_utils.store_to_database(
            df=df_result,
            db_path=financials_db_path,
            table_name=new_variable_name,
            verbose=False,
        )

    # ---- メモリ管理 ----
    # 一つのファクター処理が終わったら、使用した一時変数を削除してGC実行
    del df_base
    gc.collect()

,date,P_SYMBOL,value,variable
0,2005-08-31,0HSW-GB,2448.485005,FF_ASSETS
1,2005-09-30,0HSW-GB,2333.443621,FF_ASSETS
2,2005-10-31,0HSW-GB,2333.443621,FF_ASSETS
3,2005-11-30,0HSW-GB,2333.443621,FF_ASSETS
4,2005-12-30,0HSW-GB,NaN,FF_ASSETS
...,...,...,...,...
11077677,2025-06-30,ZURN-CH,37031.999926,FF_SALES
11077678,2025-07-31,ZURN-CH,37031.999926,FF_SALES
11077679,2025-08-29,ZURN-CH,37031.999926,FF_SALES
11077680,2025-09-30,ZURN-CH,37031.999926,FF_SALES


Factors: 100%|██████████| 14/14 [05:52<00:00, 25.15s/it]


#### 2. ファクターのランクを計算し、データベースに保存する。


In [ ]:
# -----------------------------------
# load data
# -----------------------------------
# 構成銘柄情報
df_weight = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
).assign(date=lambda x: pd.to_datetime(x["date"]))
factset_utils.process_rank_calculation_store_to_db(
    df_weight=df_weight,
    factor_list=factor_list,
    financials_db_path=financials_db_path,
    period_list=period_list,
)

🚀 処理開始: 56 件のタスク


Rank計算進捗: 100%|██████████| 56/56 [06:31<00:00,  7.00s/it]


🎉 全てのランク計算・保存が完了しました


In [ ]:
table_names = db_utils.get_table_names(financials_db_path)
display([s for s in table_names if ("QoQ" in s) or ("YoY" in s) or ("CAGR" in s)])

with sqlite3.connect(financials_db_path) as conn:
    df = pd.read_sql(
        "SELECT * FROM FF_OPER_INC_QoQ_PctRank ORDER BY 'date'",
        con=conn,
        parse_dates=["date"],
    )
    display(df.head())

['FF_EBITDA_OPER_MGN_CHANGE_QoQ',
 'FF_EBITDA_OPER_MGN_CHANGE_YoY',
 'FF_EBIT_OPER_MGN_CHANGE_QoQ',
 'FF_EBIT_OPER_MGN_CHANGE_YoY',
 'FF_NET_MGN_CHANGE_QoQ',
 'FF_NET_MGN_CHANGE_YoY',
 'FF_OPER_MGN_CHANGE_QoQ',
 'FF_OPER_MGN_CHANGE_YoY',
 'FF_PTX_MGN_CHANGE_QoQ',
 'FF_PTX_MGN_CHANGE_YoY',
 'FF_GROSS_MGN_CHANGE_QoQ',
 'FF_GROSS_MGN_CHANGE_YoY',
 'FF_ROA_CHANGE_QoQ',
 'FF_ROA_CHANGE_YoY',
 'FF_ROE_CHANGE_QoQ',
 'FF_ROE_CHANGE_YoY',
 'FF_ROIC_CHANGE_QoQ',
 'FF_ROIC_CHANGE_YoY',
 'FF_ROTC_CHANGE_QoQ',
 'FF_ROTC_CHANGE_YoY',
 'FF_ASSETS_QoQ',
 'FF_ASSETS_YoY',
 'FF_ASSETS_CAGR_3Y',
 'FF_ASSETS_CAGR_5Y',
 'FF_CAPEX_QoQ',
 'FF_CAPEX_YoY',
 'FF_CAPEX_CAGR_3Y',
 'FF_CAPEX_CAGR_5Y',
 'FF_COM_EQ_QoQ',
 'FF_COM_EQ_YoY',
 'FF_COM_EQ_CAGR_3Y',
 'FF_COM_EQ_CAGR_5Y',
 'FF_DEBT_QoQ',
 'FF_DEBT_YoY',
 'FF_DEBT_CAGR_3Y',
 'FF_DEBT_CAGR_5Y',
 'FF_DEBT_LT_QoQ',
 'FF_DEBT_LT_YoY',
 'FF_DEBT_LT_CAGR_3Y',
 'FF_DEBT_LT_CAGR_5Y',
 'FF_DEBT_ST_QoQ',
 'FF_DEBT_ST_YoY',
 'FF_DEBT_ST_CAGR_3Y',
 'FF_DEBT_ST_CAGR_5Y'

,date,P_SYMBOL,variable,value
0,2005-11-30,0HSW-GB,FF_OPER_INC_QoQ_PctRank,0.066176
1,2005-11-30,0II3.XX1-GB,FF_OPER_INC_QoQ_PctRank,0.784810
2,2005-11-30,0MDJ-GB,FF_OPER_INC_QoQ_PctRank,0.426966
3,2005-11-30,0N1N-GB,FF_OPER_INC_QoQ_PctRank,0.475124
4,2005-11-30,0N3I-GB,FF_OPER_INC_QoQ_PctRank,0.343284


### 4-2. マージン改善率


#### 1. QoQ, YoY, 3Yr, 5Yr の変化値（delta）を計算し、データベースに保存する。


In [ ]:
factor_list = [
    "FF_EBITDA_OPER_MGN",
    "FF_EBIT_OPER_MGN",
    "FF_NET_MGN",
    "FF_OPER_MGN",
    "FF_PTX_MGN",
    "FF_GROSS_MGN",
    "FF_ROA",
    "FF_ROE",
    "FF_ROIC",
    "FF_ROTC",
]

period_list = ["CHANGE_QoQ", "CHANGE_YoY", "CHANGE_3Y", "CHANGE_5Y"]

query = [
    f"SELECT `date`, `P_SYMBOL`, `variable`, `value` FROM `{table}`"
    for table in factor_list
]
query = " UNION ALL ".join(query)

In [ ]:
# ------------------------------------------------------
# load from database
# ------------------------------------------------------
with sqlite3.connect(financials_db_path) as conn:
    df_all = (
        pd.read_sql(query, con=conn, parse_dates=["date"])
        .sort_values("date", ignore_index=True)
        .assign(
            variable=lambda x: x["variable"].astype("category"),
            P_SYMBOL=lambda x: x["P_SYMBOL"].astype("category"),
        )
    ).sort_values(["variable", "P_SYMBOL", "date"], ignore_index=True)

display(df_all)
# groupby("variable")を使うことで、df_all全体を何度も走査(loc)するコストをゼロにする
# observed=True はcategory型を使う場合の高速化オプション
grouped = df_all.groupby("variable", observed=True)

# ------------------------------------------------------
# calculate growth and store to database
# ------------------------------------------------------
total_steps = len(factor_list)
for factor_name, df_factor in tqdm(grouped, total=total_steps, desc="Factors"):
    # df_factorはView(参照)の可能性があるため、計算用にコピーを作成
    # ここでメモリを食うが、factor単位なので全体コピーよりは軽い
    # かつ、ループの最後で解放される
    df_base = df_factor.copy()

    for growth in period_list:
        new_variable_name = f"{factor_name}_{growth}"
        df_result = roic_utils.calculate_margin_improvement(
            df=df_base, data_name=str(factor_name), growth_type=growth
        )

        # store to database
        db_utils.delete_table_from_database(
            db_path=financials_db_path, table_name=new_variable_name
        )
        factset_utils.store_to_database(
            df=df_result,
            db_path=financials_db_path,
            table_name=new_variable_name,
            verbose=False,
        )

    # ---- メモリ管理 ----
    # 一つのファクター処理が終わったら、使用した一時変数を削除してGC実行
    del df_base
    gc.collect()

,date,P_SYMBOL,value,variable
0,2005-08-31,0HSW-GB,9.473684,FF_EBITDA_OPER_MGN
1,2005-09-30,0HSW-GB,4.166667,FF_EBITDA_OPER_MGN
2,2005-10-31,0HSW-GB,4.166667,FF_EBITDA_OPER_MGN
3,2005-11-30,0HSW-GB,4.166667,FF_EBITDA_OPER_MGN
4,2005-12-30,0HSW-GB,NaN,FF_EBITDA_OPER_MGN
...,...,...,...,...
7912625,2025-06-30,ZURN-CH,21.581542,FF_ROTC
7912626,2025-07-31,ZURN-CH,21.581542,FF_ROTC
7912627,2025-08-29,ZURN-CH,21.581542,FF_ROTC
7912628,2025-09-30,ZURN-CH,21.581542,FF_ROTC


Factors: 100%|██████████| 10/10 [04:27<00:00, 26.74s/it]


#### 2. ファクターのランクを計算し、データベースに保存する


In [ ]:
# -----------------------------------
# load data
# -----------------------------------
# 構成銘柄情報
df_weight = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
).assign(date=lambda x: pd.to_datetime(x["date"]))
factset_utils.process_rank_calculation_store_to_db(
    df_weight=df_weight,
    factor_list=factor_list,
    financials_db_path=financials_db_path,
    period_list=period_list,
)

🚀 処理開始: 40 件のタスク


Rank計算進捗: 100%|██████████| 40/40 [05:10<00:00,  7.75s/it]


🎉 全てのランク計算・保存が完了しました


### 4-3. Composite Growth Factor


#### 1. データロード&欠損値確認 -> 欠損値はセクター中央値で補完


In [ ]:
# growthファクター計算のためのディスクリプターを指定
factor_list = [
    "FF_SALES_CAGR_3Y_PctRank",
    "FF_OPER_INC_CAGR_3Y_PctRank",
    "FF_OPER_MGN_CHANGE_3Y_PctRank",
    "FF_GROSS_MGN_CHANGE_3Y_PctRank",
    "FF_NET_MGN_CHANGE_3Y_PctRank",
]

# -----------------------------------
# load data
# -----------------------------------

df_factor = factset_utils.load_financial_data(
    financials_db_path=financials_db_path, factor_list=factor_list
)

df_weight = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
)
df = (
    factset_utils.merge_idx_constituents_and_financials(
        df_weight=df_weight, df_factor=df_factor
    )
    .fillna(np.nan)
    .dropna(subset=factor_list, how="all")
    .dropna(subset=["Weight (%)"], ignore_index=True)
)
display(df.head())

factset_utils.check_missing_value_and_fill_by_sector_median(
    df=df, factor_list=factor_list
)

# -----------------------------------
# データ確認
# -----------------------------------
display(df)
g = pd.DataFrame(df.groupby(["date"])["Weight (%)"].agg("sum"))
g_count = pd.DataFrame(df.groupby(["date"])["SEDOL"].count())
g_merged = pd.merge(g, g_count, left_index=True, right_index=True)

display(g_merged)

,date,P_SYMBOL,SEDOL,Asset ID,FG_COMPANY_NAME,GICS Sector,GICS Industry Group,Weight (%),FF_GROSS_MGN_CHANGE_3Y_PctRank,FF_NET_MGN_CHANGE_3Y_PctRank,FF_OPER_INC_CAGR_3Y_PctRank,FF_OPER_MGN_CHANGE_3Y_PctRank,FF_SALES_CAGR_3Y_PctRank
0,2008-08-31,0MDJ-GB,B2PF6M7,UKIBBF1,Cadbury PLC,Consumer Staples,Food Beverage & Tobacco,0.070948,NaN,0.337079,0.034483,0.114943,0.072165
1,2008-08-31,0N3I-GB,0896265,UKIDBM1,Tomkins PLC,Industrials,Capital Goods,0.010869,0.351145,0.010929,0.408284,0.136095,0.793970
2,2008-08-31,0P7J-GB,0028262,UKIAPY1,Amec Foster Wheeler plc,Energy,Energy,0.023192,0.644444,0.954545,0.527473,0.956044,0.019608
3,2008-08-31,1-HK,6190273,HKGAAE1,CKハチソン・ホールディングス,Financials,Real Estate,0.090836,0.235955,0.117886,0.875000,0.470852,0.925170
4,2008-08-31,10-HK,6408352,HKGAGG1,Hang Lung Group Limited,Financials,Real Estate,0.016922,0.584270,0.743902,NaN,NaN,0.112245


📋 欠損値の状況（補完前）
FF_SALES_CAGR_3Y_PctRank                     :    365件 (  0.1%)
FF_OPER_INC_CAGR_3Y_PctRank                  : 29,077件 ( 11.2%)
FF_OPER_MGN_CHANGE_3Y_PctRank                : 29,264件 ( 11.2%)
FF_GROSS_MGN_CHANGE_3Y_PctRank               : 47,146件 ( 18.1%)
FF_NET_MGN_CHANGE_3Y_PctRank                 : 48,076件 ( 18.5%)

⏳ セクター中央値で補完中...

📋 欠損値の状況（セクター中央値補完後）
✅ FF_SALES_CAGR_3Y_PctRank                     :      0件 (  0.0%) | 補完: 365件
✅ FF_OPER_INC_CAGR_3Y_PctRank                  :      0件 (  0.0%) | 補完: 29,077件
✅ FF_OPER_MGN_CHANGE_3Y_PctRank                :      0件 (  0.0%) | 補完: 29,264件
✅ FF_GROSS_MGN_CHANGE_3Y_PctRank               :      0件 (  0.0%) | 補完: 47,146件
✅ FF_NET_MGN_CHANGE_3Y_PctRank                 :      0件 (  0.0%) | 補完: 48,076件

✅ 最終欠損値チェック
✅ FF_SALES_CAGR_3Y_PctRank                     : 欠損なし
✅ FF_OPER_INC_CAGR_3Y_PctRank                  : 欠損なし
✅ FF_OPER_MGN_CHANGE_3Y_PctRank                : 欠損なし
✅ FF_GROSS_MGN_CHANGE_3Y_PctRank               : 欠損なし


,date,P_SYMBOL,SEDOL,Asset ID,FG_COMPANY_NAME,GICS Sector,GICS Industry Group,Weight (%),FF_GROSS_MGN_CHANGE_3Y_PctRank,FF_NET_MGN_CHANGE_3Y_PctRank,FF_OPER_INC_CAGR_3Y_PctRank,FF_OPER_MGN_CHANGE_3Y_PctRank,FF_SALES_CAGR_3Y_PctRank
0,2008-08-31,0MDJ-GB,B2PF6M7,UKIBBF1,Cadbury PLC,Consumer Staples,Food Beverage & Tobacco,0.070948,0.505814,0.337079,0.034483,0.114943,0.072165
1,2008-08-31,0N3I-GB,0896265,UKIDBM1,Tomkins PLC,Industrials,Capital Goods,0.010869,0.351145,0.010929,0.408284,0.136095,0.793970
2,2008-08-31,0P7J-GB,0028262,UKIAPY1,Amec Foster Wheeler plc,Energy,Energy,0.023192,0.644444,0.954545,0.527473,0.956044,0.019608
3,2008-08-31,1-HK,6190273,HKGAAE1,CKハチソン・ホールディングス,Financials,Real Estate,0.090836,0.235955,0.117886,0.875000,0.470852,0.925170
4,2008-08-31,10-HK,6408352,HKGAGG1,Hang Lung Group Limited,Financials,Real Estate,0.016922,0.584270,0.743902,0.502232,0.502242,0.112245
...,...,...,...,...,...,...,...,...,...,...,...,...,...
260260,2025-10-31,ZAL-DE,BQV0SV7,GERZQZ1,ザランド,Consumer Discretionary,Consumer Discretionary Distribution & Retail,0.008099,0.524390,0.507937,0.507042,0.507042,0.755814
260261,2025-10-31,ZBH-US,2783815,USA4JT1,ジンマー・バイオメット・ホールディングス,Health Care,Health Care Equipment & Services,0.025627,0.595506,0.511905,0.510870,0.510870,0.438776
260262,2025-10-31,ZBRA-US,2989356,USAP8H1,ゼブラ・テクノロジーズ・コーポレーション Class A,Information Technology,Technology Hardware & Equipment,0.017637,0.819277,0.136364,0.366197,0.633803,0.190476
260263,2025-10-31,ZTS-US,B95WG16,USBANZ1,ゾエティス Class A,Health Care,Pharmaceuticals Biotechnology & Life Sciences,0.082630,0.707865,0.666667,0.597826,0.608696,0.459184


,Weight (%),SEDOL
date,,
2008-08-31,96.671657,1309
2008-09-30,96.652476,1305
2008-10-31,96.929219,1305
2008-11-30,96.983162,1277
2008-12-31,96.490301,1272
...,...,...
2025-06-30,99.285484,1123
2025-07-31,99.335974,1122
2025-08-31,99.333187,1123


#### 2. Composite Growth Factor 計算


In [ ]:
# -----------------------------------
# Composite Growth Factorの計算
# -----------------------------------
# ウェイト設定
blend_weight = {
    "FF_SALES_CAGR_3Y_PctRank": 0.20,
    "FF_OPER_INC_CAGR_3Y_PctRank": 0.20,
    "FF_OPER_MGN_CHANGE_3Y_PctRank": 0.20,
    "FF_GROSS_MGN_CHANGE_3Y_PctRank": 0.20,
    "FF_NET_MGN_CHANGE_3Y_PctRank": 0.20,
}
factor_name = "Factor_Composite_Growth"
df = factset_utils.create_factor(
    df=df, factor_name=factor_name, blend_weight=blend_weight
)
# -----------------------------------
# Store to database
# -----------------------------------
for variable in ["Factor_Composite_Growth_Score", "Factor_Composite_Growth_Score_Rank"]:
    df_slice = (
        df[["date", "P_SYMBOL", variable]]
        .assign(variable=variable, date=lambda x: pd.to_datetime(x["date"]))
        .rename(columns={variable: "value"})
    )
    db_utils.delete_table_from_database(db_path=financials_db_path, table_name=variable)
    factset_utils.store_to_database(
        df=df_slice,
        db_path=financials_db_path,
        table_name=variable,
        verbose=True,
    )


📊 Composite Growth Factor 計算
ウェイト設定:
  FF_SALES_CAGR_3Y_PctRank                     : 20.0%
  FF_OPER_INC_CAGR_3Y_PctRank                  : 20.0%
  FF_OPER_MGN_CHANGE_3Y_PctRank                : 20.0%
  FF_GROSS_MGN_CHANGE_3Y_PctRank               : 20.0%
  FF_NET_MGN_CHANGE_3Y_PctRank                 : 20.0%

✅ Composite Growth Score 計算完了
   平均: 0.5049
   標準偏差: 0.1889
   最小値: 0.0098
   最大値: 1.0000

📊 Composite Growth Rank 分布:
  rank5: 52,142件 ( 20.0%)
  rank4: 52,007件 ( 20.0%)
  rank3: 52,015件 ( 20.0%)
  rank2: 52,008件 ( 20.0%)
  rank1: 52,093件 ( 20.0%)

🎯 最終データサンプル:


,date,P_SYMBOL,SEDOL,GICS Sector,Factor_Composite_Growth_Score,Factor_Composite_Growth_Score_Rank
260245,2025-10-31,WTB-GB,B1KJJ40,Consumer Discretionary,0.604619,rank2
260246,2025-10-31,WTC-AU,BZ8GX83,Information Technology,0.429761,rank4
260247,2025-10-31,WTRG-US,BLCF3J9,Utilities,0.589394,rank2
260248,2025-10-31,WTW-US,BDB6Q21,Financials,0.717710,rank1
260249,2025-10-31,WY-US,2958936,Real Estate,0.053492,rank5
260250,2025-10-31,X-CA,B8KH5G7,Financials,0.561373,rank3
260251,2025-10-31,XEL-US,2614807,Utilities,0.396061,rank4
260252,2025-10-31,XOM-US,2326618,Energy,0.490537,rank3
260253,2025-10-31,XRO-AU,B8P4LP4,Information Technology,0.763432,rank1
260254,2025-10-31,XYL-US,B3P2CN8,Industrials,0.637115,rank2



📊 セクター別の平均Growth Score（最新日付）:


GICS Sector
Energy                    0.511042
Communication Services    0.510529
Real Estate               0.508650
Utilities                 0.508333
Consumer Staples          0.507167
Materials                 0.507054
Consumer Discretionary    0.506787
Information Technology    0.506727
Health Care               0.505508
Financials                0.503845
Industrials               0.502700
Name: Factor_Composite_Growth_Score, dtype: float64

既存の 0 行との重複をチェックしました。260250 行を新たに追加します。
  -> Factor_Composite_Growth_Score: データの書き込みが完了しました。
既存の 0 行との重複をチェックしました。260250 行を新たに追加します。
  -> Factor_Composite_Growth_Score_Rank: データの書き込みが完了しました。


### 4-4. Valuation（Bloomberg） -> ⚠️ データ不足のためスキップ

`BEST_EPS`と`TRAIL_12M_EPS_BEF_XO_ITEM`の QoQ, YoY, 3Yr CAGR, 5Yr CAGR の値を計算し、データベースに保存する。


In [ ]:
data_list = ["BEST_PE_RATIO", "BEST_EPS", "PE_RATIO", "TRAIL_12M_EPS_BEF_XO_ITEM"]

blp = bloomberg_utils.BlpapiCustom()

with sqlite3.connect(bloomberg_valuation_db_path) as conn:
    for data in tqdm(data_list):
        df = (
            pd.read_sql(f"SELECT * FROM `{data}`", con=conn, parse_dates=["Date"])
            .sort_values("Date", ignore_index=True)
            .rename(columns={"Date": "date", "SEDOL": "P_SYMBOL"})
        )  # calculate_growth関数を使うために一時的にリネーム

        for growth in ["QoQ", "YoY", "CAGR_3Y", "CAGR_5Y"]:
            df_growth = df.copy()
            new_variable = f"{data}_{growth}"

            df_growth = roic_utils.calculate_growth(
                df=df_growth, data_name=data, growth_type=growth
            ).rename(
                columns={"date": "Date", "P_SYMBOL": "SEDOL"}
            )  # 元のカラム名に戻す

            db_utils.delete_table_from_database(
                db_path=bloomberg_valuation_db_path, table_name=new_variable
            )

            blp.store_to_database(
                df=df_growth,
                db_path=bloomberg_valuation_db_path,
                table_name=new_variable,
                primary_keys=["date", "SEDOL", "variable"],
            )

  0%|          | 0/4 [00:00<?, ?it/s]

✅ 保存完了。テーブル 'BEST_PE_RATIO_QoQ' に 55043 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'BEST_PE_RATIO_YoY' に 38534 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'BEST_PE_RATIO_CAGR_3Y' に 10608 行を処理しました (IGNORE)。


 25%|██▌       | 1/4 [00:13<00:39, 13.07s/it]

✅ 保存完了。テーブル 'BEST_PE_RATIO_CAGR_5Y' に 889 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'BEST_EPS_QoQ' に 61342 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'BEST_EPS_YoY' に 41722 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'BEST_EPS_CAGR_3Y' に 7895 行を処理しました (IGNORE)。


 50%|█████     | 2/4 [00:25<00:25, 12.51s/it]

✅ 保存完了。テーブル 'BEST_EPS_CAGR_5Y' に 122 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'PE_RATIO_QoQ' に 512335 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'PE_RATIO_YoY' に 485879 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'PE_RATIO_CAGR_3Y' に 419875 行を処理しました (IGNORE)。


 75%|███████▌  | 3/4 [02:43<01:09, 69.83s/it]

✅ 保存完了。テーブル 'PE_RATIO_CAGR_5Y' に 359299 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'TRAIL_12M_EPS_BEF_XO_ITEM_QoQ' に 161910 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'TRAIL_12M_EPS_BEF_XO_ITEM_YoY' に 136498 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'TRAIL_12M_EPS_BEF_XO_ITEM_CAGR_3Y' に 61627 行を処理しました (IGNORE)。


100%|██████████| 4/4 [03:25<00:00, 51.43s/it]

✅ 保存完了。テーブル 'TRAIL_12M_EPS_BEF_XO_ITEM_CAGR_5Y' に 32107 行を処理しました (IGNORE)。


In [ ]:
with sqlite3.connect(bloomberg_valuation_db_path) as conn:
    df = pd.read_sql("SELECT * FROM BEST_EPS", con=conn, parse_dates=["Date"])
    display(df)

,Date,SEDOL,variable,value
0,2000-01-31,2206301,BEST_EPS,0.820
1,2000-01-31,2213981,BEST_EPS,3.653
2,2000-01-31,2226536,BEST_EPS,0.033
3,2000-01-31,2350651,BEST_EPS,0.230
4,2000-01-31,2552275,BEST_EPS,0.220
...,...,...,...,...
68844,2025-10-31,BZ8GX83,BEST_EPS,0.343
68845,2025-10-31,BZ8VJQ8,BEST_EPS,2.270
68846,2025-10-31,BZBW6G7,BEST_EPS,0.196
68847,2025-10-31,BZBYG74,BEST_EPS,0.216


In [ ]:
tables = db_utils.get_table_names(bloomberg_valuation_db_path)
display(tables)

['BEST_PE_RATIO',
 'BEST_EPS',
 'PE_RATIO',
 'TRAIL_12M_EPS_BEF_XO_ITEM',
 'BEST_PE_RATIO_QoQ',
 'BEST_PE_RATIO_YoY',
 'BEST_PE_RATIO_CAGR_3Y',
 'BEST_PE_RATIO_CAGR_5Y',
 'BEST_EPS_QoQ',
 'BEST_EPS_YoY',
 'BEST_EPS_CAGR_3Y',
 'BEST_EPS_CAGR_5Y',
 'PE_RATIO_QoQ',
 'PE_RATIO_YoY',
 'PE_RATIO_CAGR_3Y',
 'PE_RATIO_CAGR_5Y',
 'TRAIL_12M_EPS_BEF_XO_ITEM_QoQ',
 'TRAIL_12M_EPS_BEF_XO_ITEM_YoY',
 'TRAIL_12M_EPS_BEF_XO_ITEM_CAGR_3Y',
 'TRAIL_12M_EPS_BEF_XO_ITEM_CAGR_5Y']

In [ ]:
# 構成銘柄情報
query = f"""
    SELECT
        `date`, `P_SYMBOL`, `SEDOL`, `FG_COMPANY_NAME`, `Asset ID`, `GICS Sector`, `Weight (%)`
    FROM
        {UNIVERSE_CODE}
"""
with sqlite3.connect(factset_index_db_path) as conn:
    df_weight = pd.read_sql(query, parse_dates=["date"], con=conn)

# ファクター値
factor_list = ["BEST_EPS", "BEST_PE_RATIO"]

with sqlite3.connect(bloomberg_valuation_db_path) as conn:
    # total引数を追加
    for factor in tqdm(factor_list):
        # データベースから呼び出し
        df = (
            pd.read_sql(
                f"SELECT `Date`, `SEDOL`, `value` FROM `{factor}`",
                con=conn,
                parse_dates=["Date"],
            )
            .rename(columns={"Date": "date"})
            .assign(
                date=lambda row: pd.to_datetime(row["date"])
                + pd.tseries.offsets.MonthEnd(0)
            )
            .sort_values("date", ignore_index=True)
            .rename(columns={"value": factor})
        )

        # merge: 構成銘柄情報とファクター
        df = (
            pd.merge(df_weight, df, on=["date", "SEDOL"], how="outer")
            .drop_duplicates(subset=["date", "SEDOL"])
            .dropna(
                subset=["Weight (%)", factor],
                how="any",
                axis=0,
                ignore_index=True,
            )
        )

        g = df.groupby(["date"])["Weight (%)"].agg("sum").to_frame()
        display(g.tail(50))

  0%|          | 0/2 [00:00<?, ?it/s]

,Weight (%)
date,
2021-09-30,21.202650
2021-10-31,36.429975
2021-11-30,38.705860
2021-12-31,23.446075
2022-01-31,42.453669
2022-02-28,35.490503
2022-03-31,31.115912
2022-04-30,43.708097
2022-05-31,40.849117


 50%|█████     | 1/2 [00:01<00:01,  1.11s/it]

,Weight (%)
date,
2021-09-30,24.751970
2021-10-31,30.705197
2021-11-30,39.116527
2021-12-31,40.400506
2022-01-31,42.623696
2022-02-28,45.424370
2022-03-31,47.750144
2022-04-30,49.021961
2022-05-31,50.595893


100%|██████████| 2/2 [00:02<00:00,  1.07s/it]


In [ ]:
# 構成銘柄情報
query = f"""
    SELECT
        `date`, `P_SYMBOL`, `SEDOL`, `FG_COMPANY_NAME`, `Asset ID`, `GICS Sector`, `Weight (%)`
    FROM
        {UNIVERSE_CODE}
"""
with sqlite3.connect(factset_index_db_path) as conn:
    df_weight = pd.read_sql(query, parse_dates=["date"], con=conn)

# ファクター値
factor_list = ["BEST_EPS", "BEST_PE_RATIO"]
period_list = ["QoQ", "YoY", "CAGR_3Y", "CAGR_5Y"]
total_iterations = len(factor_list) * len(period_list)
print(f"処理総数: {total_iterations} 回")

with sqlite3.connect(bloomberg_valuation_db_path) as conn:
    # total引数を追加
    for factor, periods in tqdm(
        itertools.product(factor_list, period_list),
        total=total_iterations,
        desc="ファクター処理進捗",
    ):
        factor_growth = f"{factor}_{periods}"
        # データベースから呼び出し
        df = (
            pd.read_sql(
                f"SELECT `Date`, `SEDOL`, `value` FROM `{factor_growth}`",
                con=conn,
                parse_dates=["Date"],
            )
            .rename(columns={"Date": "date"})
            .assign(
                date=lambda row: pd.to_datetime(row["date"])
                + pd.tseries.offsets.MonthEnd(0)
            )
            .sort_values("date", ignore_index=True)
            .rename(columns={"value": factor_growth})
        )

        # merge: 構成銘柄情報とファクター
        df = (
            pd.merge(df_weight, df, on=["date", "SEDOL"], how="outer")
            .drop_duplicates(subset=["date", "SEDOL"])
            .dropna(
                subset=["Weight (%)", factor_growth],
                how="any",
                axis=0,
                ignore_index=True,
            )
        )
        display(df)

処理総数: 8 回


ファクター処理進捗:   0%|          | 0/8 [00:00<?, ?it/s]

,date,P_SYMBOL,FG_COMPANY_NAME,Asset ID,GICS Sector,Weight (%),BEST_EPS_QoQ


ファクター処理進捗:  12%|█▎        | 1/8 [00:01<00:08,  1.16s/it]

,date,P_SYMBOL,FG_COMPANY_NAME,Asset ID,GICS Sector,Weight (%),BEST_EPS_YoY


...

ファクター処理進捗:  75%|███████▌  | 6/8 [00:04<00:01,  1.28it/s]

,date,P_SYMBOL,FG_COMPANY_NAME,Asset ID,GICS Sector,Weight (%),BEST_PE_RATIO_CAGR_3Y


ファクター処理進捗:  88%|████████▊ | 7/8 [00:05<00:00,  1.49it/s]

,date,P_SYMBOL,FG_COMPANY_NAME,Asset ID,GICS Sector,Weight (%),BEST_PE_RATIO_CAGR_5Y


ファクター処理進捗: 100%|██████████| 8/8 [00:05<00:00,  1.49it/s]


### 4-5. ROIC Label Factor（Factset）

-   ROIC(ROE) + Security Code
    -   セクター中立
    -   金融セクターのみ ROIC の代わりに ROE を使用（ただしデータフレームのカラム名は ROIC で表記）


In [ ]:
# ----------------------------------------
# 1. get ROIC and ROE data
# 2. get security info(Index constituents)
# ----------------------------------------
df_roic_and_roe = factset_utils.load_financial_data(
    financials_db_path=financials_db_path, factor_list=["FF_ROIC", "FF_ROE"]
)
security_info = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
)

# ----------------------------------------
# merge
# ----------------------------------------
df_roic_merged = (
    factset_utils.merge_idx_constituensts_and_financials(
        df_weight=security_info, df_factor=df_roic_and_roe
    )
    .assign(
        ROIC=lambda x: np.where(
            x["GICS Sector"] == "Financials", x["FF_ROE"], x["FF_ROIC"]
        )
    )
    .dropna(subset=["Weight (%)", "ROIC"], how="any", ignore_index=True)
    .drop(columns=["FF_ROIC", "FF_ROE"])
)

# ----------------------------------------
# sector中立でROICをランキング
# ----------------------------------------
year_period = 3  # year_periodまでさかのぼってROICの移行を見る

df_roic_merged = roic_utils.add_factor_rank_cols(
    df_roic_merged, factor_name="ROIC"
).rename(columns={"P_SYMBOL": "Symbol"})
df_roic_merged = roic_utils.add_shifted_factor_cols_month(
    df_roic_merged,
    factor_name="ROIC_Rank",
    shift_month=list(range(1, int(year_period * 12) + 1)),
    shift_direction="Past",
).rename(columns={"Symbol": "P_SYMBOL"})


# ----------------------------------------
# ROICラベルを付与
# ----------------------------------------
roic_label_name = f"ROIC_label_Past{year_period}Y"
df_roic_merged[roic_label_name] = df_roic_merged.apply(
    lambda row: roic_utils.test_assign_roic_label(
        row=row,
        freq="annual",
        shift_direction="Past",
        year_period=year_period,
        judge_by_slope=False,
    ),
    axis=1,
)

df = df_roic_merged.copy()
df = (
    df[["date", "P_SYMBOL", roic_label_name]]
    .rename(columns={roic_label_name: "value"})
    .dropna(subset=["value"], ignore_index=True)
    .assign(variable=roic_label_name, date=lambda row: pd.to_datetime(row["date"]))
)
display(df.tail(10))

# ----------------------------------------
# データベース保存
# ----------------------------------------
factset_utils.store_to_database(
    df=df, db_path=financials_db_path, table_name=roic_label_name
)

,date,P_SYMBOL,value,variable
220535,2025-10-31,XYZ-US,drop to low,ROIC_label_Past3Y
220536,2025-10-31,YAR-NO,move to high,ROIC_label_Past3Y
220537,2025-10-31,YUM-US,remain high,ROIC_label_Past3Y
220538,2025-10-31,Z-US,remain low,ROIC_label_Past3Y
220539,2025-10-31,Z74-SG,others,ROIC_label_Past3Y
220540,2025-10-31,ZAL-DE,remain low,ROIC_label_Past3Y
220541,2025-10-31,ZBH-US,others,ROIC_label_Past3Y
220542,2025-10-31,ZBRA-US,others,ROIC_label_Past3Y
220543,2025-10-31,ZTS-US,remain high,ROIC_label_Past3Y
220544,2025-10-31,ZURN-CH,remain high,ROIC_label_Past3Y


テーブル 'ROIC_label_Past3Y' に追加すべき新しいデータはありませんでした。スキップします。


In [ ]:
df = df_roic_merged.copy()

# --- label count ---
roic_count = pd.pivot(
    pd.DataFrame(
        df.groupby(["date", "GICS Sector", roic_label_name])["P_SYMBOL"].count()
    ).reset_index(),
    index=["date", "GICS Sector"],
    columns=roic_label_name,
).reset_index()
display(roic_count.loc[roic_count["GICS Sector"] == "Information Technology"])

roic_count = pd.pivot(
    pd.DataFrame(
        df.groupby(["date", "GICS Sector", "ROIC_Rank"])["P_SYMBOL"].count()
    ).reset_index(),
    index=["date", "GICS Sector"],
    columns="ROIC_Rank",
).reset_index()
display(roic_count.loc[roic_count["GICS Sector"] == "Information Technology"])

# --- weight check ---
weight_total_count = (
    df.groupby(["date"])["Weight (%)"]
    .agg(["count", "sum"])
    .rename(columns={"count": "Num of Securities", "sum": "Total Weight (%)"})
).sort_index()

weight_sector_count = (
    df.groupby(["date", "GICS Sector"])["Weight (%)"]
    .agg(["count", "sum"])
    .rename(columns={"count": "Num of Securities", "sum": "Total Weight (%)"})
).sort_index()
display(weight_total_count)
display(weight_sector_count)

roic_label_count = (
    df.groupby(["date", roic_label_name])["Weight (%)"]
    .agg(["count", "sum"])
    .rename(columns={"count": "Num of Securities", "sum": "Total Weight (%)"})
).sort_index()
display(roic_label_count)

date             GICS Sector    P_SYMBOL               \
ROIC_label_Past3Y                                    drop to low move to high   
11                2008-08-31  Information Technology         7.0          8.0   
21                2008-09-30  Information Technology         5.0          8.0   
31                2008-10-31  Information Technology         6.0          9.0   
41                2008-11-30  Information Technology         6.0         10.0   
51                2008-12-31  Information Technology         5.0         10.0   
...                      ...                     ...         ...          ...   
2136              2025-06-30  Information Technology        12.0         15.0   
2147              2025-07-31  Information Technology        13.0         16.0   
2158              2025-08-31  Information Technology        14.0         15.0   
2169              2025-09-30  Information Technology        11.0         13.0   
2180              2025-10-31  Information Technology         9.0         12.0   

                                                 
ROIC_label_Past3Y others remain high remain low  
11                  17.0        22.0       10.0  
21                  17.0        23.0       13.0  
31                  18.0        21.0       12.0  
41                  16.0        23.0       10.0  
51                  20.0        22.0        9.0  
...                  ...         ...        ...  
2136                28.0        28.0       20.0  
2147                27.0        28.0       18.0  
2158                25.0        29.0       18.0  
2169                25.0        28.0       20.0  
2180                22.0        21.0       11.0  

[207 rows x 7 columns]

date             GICS Sector P_SYMBOL                        
ROIC_Rank                                       rank1 rank2 rank3 rank4 rank5
7         2005-08-31  Information Technology       25    25    24    25    25
17        2005-09-30  Information Technology       25    25    25    25    25
27        2005-10-31  Information Technology       25    25    25    25    26
37        2005-11-30  Information Technology       25    25    25    25    26
47        2005-12-31  Information Technology       25    25    25    25    25
...              ...                     ...      ...   ...   ...   ...   ...
2492      2025-06-30  Information Technology       23    22    23    22    23
2503      2025-07-31  Information Technology       23    22    22    22    23
2514      2025-08-31  Information Technology       23    22    22    22    23
2525      2025-09-30  Information Technology       22    21    22    21    22
2536      2025-10-31  Information Technology       17    16    16    16    17

[243 rows x 7 columns]

,Num of Securities,Total Weight (%)
date,,
2005-08-31,1296,91.341262
2005-09-30,1291,91.723118
2005-10-31,1290,91.461135
2005-11-30,1284,91.742185
2005-12-31,1318,95.426320
...,...,...
2025-06-30,1127,99.065370
2025-07-31,1126,99.203499
2025-08-31,1127,99.198368


Num of Securities  Total Weight (%)
date       GICS Sector                                                
2005-08-31 Communication Services                 40          4.554126
           Consumer Discretionary                231         10.119443
           Consumer Staples                       91          8.564437
           Energy                                 73         10.562697
           Financials                            280         21.092156
...                                              ...               ...
2025-10-31 Industrials                           195          9.800280
           Information Technology                 82         18.765398
           Materials                              78          3.004112
           Real Estate                            61          1.787467
           Utilities                              66          2.674488

[2540 rows x 2 columns]

Num of Securities  Total Weight (%)
date       ROIC_label_Past3Y                                     
2008-04-30 remain high                        2          0.402252
2008-05-31 remain high                        2          0.411722
2008-06-30 others                             2          0.494384
2008-07-31 others                             2          0.470749
2008-08-31 drop to low                      106          8.383848
...                                         ...               ...
2025-10-31 drop to low                      148         12.400099
           move to high                     149          9.408305
           others                           293         18.037149
           remain high                      201         34.724012
           remain low                       162          7.284462

[1039 rows x 2 columns]

### 4-6. ROIC 分位移動(⚠️not completed)

-   3 年前の ROIC5 分位 -> 現在の ROIC5 分位への移動をラベリング
-   Financials セクターは ROIC の代わりに ROE 使用


In [ ]:
df_weight = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
)

df_factor = factset_utils.load_financial_data(
    financials_db_path=financials_db_path, factor_list=["FF_ROIC_Rank", "FF_ROE_Rank"]
)

df_roic_rank = (
    factset_utils.merge_idx_constituensts_and_financials(
        df_weight=df_weight, df_factor=df_factor
    )
    .assign(
        ROIC_Rank=lambda x: np.where(
            x["GICS Sector"] == "Financials", x["FF_ROE_Rank"], x["FF_ROIC_Rank"]
        )
    )
    .drop(columns=["FF_ROIC_Rank", "FF_ROE_Rank"])
    .rename(columns={"SEDOL": "Symbol"})
)
df_roic_rank = (
    roic_utils.add_shifted_factor_cols_month(
        df_roic_rank,
        factor_name="ROIC_Rank",
        shift_month=[36],
        shift_direction="Past",
    )
    .rename(columns={"Symbol": "SEDOL"})
    .dropna(subset=["ROIC_Rank", "ROIC_Rank_36MAgo"], how="any")
    .sort_values(["SEDOL", "date"], ignore_index=True)
)

display(df_roic_rank)

,date,P_SYMBOL,SEDOL,Asset ID,FG_COMPANY_NAME,GICS Sector,GICS Industry Group,Weight (%),ROIC_Rank,ROIC_Rank_36MAgo
0,2015-06-30,ADN-GB,0003128,UKIDUK1,Aberdeen Asset Management PLC,Financials,Financial Services,0.019301,rank1,rank1
1,2015-07-31,ADN-GB,0003128,UKIDUK1,Aberdeen Asset Management PLC,Financials,Financial Services,0.016965,rank1,rank1
2,2015-08-31,ADN-GB,0003128,UKIDUK1,Aberdeen Asset Management PLC,Financials,Financial Services,0.015731,rank1,rank1
3,2015-09-30,ADN-GB,0003128,UKIDUK1,Aberdeen Asset Management PLC,Financials,Financial Services,0.014948,rank1,rank1
4,2015-10-31,ADN-GB,0003128,UKIDUK1,Aberdeen Asset Management PLC,Financials,Financial Services,0.016557,rank1,rank1
...,...,...,...,...,...,...,...,...,...,...
206145,2025-06-30,DHER-DE,BZCNB42,GER2BR1,デリバリー・ヒーロー,Consumer Discretionary,Consumer Services,0.007228,rank5,rank5
206146,2025-07-31,DHER-DE,BZCNB42,GER2BR1,デリバリー・ヒーロー,Consumer Discretionary,Consumer Services,0.007958,rank5,rank5
206147,2025-08-31,DHER-DE,BZCNB42,GER2BR1,デリバリー・ヒーロー,Consumer Discretionary,Consumer Services,0.006908,rank5,rank5
206148,2025-09-30,DHER-DE,BZCNB42,GER2BR1,デリバリー・ヒーロー,Consumer Discretionary,Consumer Services,0.007233,rank5,rank5


In [ ]:
g = pd.DataFrame(df_roic_rank.groupby("date")["Weight (%)"].agg("sum"))
display(g)

g_count = (
    pd.DataFrame(df_roic_rank.groupby(["date", "GICS Sector"])["SEDOL"].count())
    .reset_index()
    .pivot(index=["date"], columns="GICS Sector", values="SEDOL")
)
display(g_count)

,Weight (%)
date,
2008-08-31,74.801052
2008-09-30,76.101417
2008-10-31,76.204146
2008-11-30,76.204061
2008-12-31,80.869285
...,...
2025-06-30,93.851450
2025-07-31,94.125253
2025-08-31,93.974811


GICS Sector,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
date,,,,,,,,,,,
2008-08-31,30.0,131.0,63.0,57.0,193.0,59.0,124.0,69.0,63.0,NaN,44.0
2008-09-30,31.0,126.0,61.0,58.0,190.0,63.0,123.0,70.0,65.0,NaN,41.0
2008-10-31,31.0,127.0,61.0,59.0,190.0,63.0,122.0,70.0,65.0,NaN,42.0
2008-11-30,31.0,117.0,60.0,59.0,183.0,62.0,119.0,68.0,65.0,NaN,42.0
2008-12-31,28.0,131.0,64.0,58.0,183.0,67.0,121.0,72.0,65.0,NaN,48.0
...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,50.0,92.0,79.0,43.0,179.0,96.0,172.0,99.0,65.0,60.0,64.0
2025-07-31,50.0,92.0,79.0,43.0,179.0,97.0,172.0,98.0,65.0,60.0,64.0
2025-08-31,49.0,91.0,77.0,43.0,179.0,95.0,172.0,97.0,64.0,60.0,64.0


In [ ]:
g_roic_rank_shift = (
    pd.DataFrame(
        df_roic_rank.groupby(["date", "ROIC_Rank", "ROIC_Rank_36MAgo"])["SEDOL"].count()
    )
    .reset_index()
    .rename(columns={"SEDOL": "n_SEDOL"})
)
g_sedol_count_present_rank = (
    pd.DataFrame(g_roic_rank_shift.groupby(["date", "ROIC_Rank"])["n_SEDOL"].sum())
    .reset_index()
    .rename(columns={"n_SEDOL": "n_SEDOL_ROIC_Rank"})
)

g_sedol_count_past_rank = (
    pd.DataFrame(
        g_roic_rank_shift.groupby(["date", "ROIC_Rank_36MAgo"])["n_SEDOL"].sum()
    )
    .reset_index()
    .rename(columns={"n_SEDOL": "n_SEDOL_ROIC_Rank_36MAgo"})
)

g_roic_rank_shift = pd.merge(
    g_roic_rank_shift, g_sedol_count_present_rank, on=["date", "ROIC_Rank"], how="left"
)
g_roic_rank_shift = pd.merge(
    g_roic_rank_shift,
    g_sedol_count_past_rank,
    on=["date", "ROIC_Rank_36MAgo"],
    how="left",
)

g_roic_rank_shift = g_roic_rank_shift.assign(
    n_SEDOL_ROIC_Rank_pct=lambda x: x["n_SEDOL"].div(x["n_SEDOL_ROIC_Rank"]),
    n_SEDOL_ROIC_Rank_36MAgo_pct=lambda x: x["n_SEDOL"].div(
        x["n_SEDOL_ROIC_Rank_36MAgo"]
    ),
)
pd.options.display.precision = 2
display(g_roic_rank_shift[g_roic_rank_shift["ROIC_Rank"] == "rank1"].tail(50))

,date,ROIC_Rank,ROIC_Rank_36MAgo,n_SEDOL,n_SEDOL_ROIC_Rank,n_SEDOL_ROIC_Rank_36MAgo,n_SEDOL_ROIC_Rank_pct,n_SEDOL_ROIC_Rank_36MAgo_pct
4925,2025-01-31,rank1,rank1,126,208,225,0.61,0.56
4926,2025-01-31,rank1,rank2,42,208,210,0.20,0.20
4927,2025-01-31,rank1,rank3,18,208,217,0.09,0.08
4928,2025-01-31,rank1,rank4,10,208,206,0.05,0.05
4929,2025-01-31,rank1,rank5,12,208,188,0.06,0.06
4950,2025-02-28,rank1,rank1,126,209,224,0.60,0.56
4951,2025-02-28,rank1,rank2,42,209,210,0.20,0.20
4952,2025-02-28,rank1,rank3,18,209,218,0.09,0.08
4953,2025-02-28,rank1,rank4,10,209,205,0.05,0.05
4954,2025-02-28,rank1,rank5,13,209,191,0.06,0.07


### 4-7. Size Factor


#### 1. QoQ、YoY、3Yr CAGR, 5Y CAGR を計算してデータベースに保存する。


In [ ]:
factor_list = ["FF_SALES", "FF_ASSETS", "FF_COM_EQ", "FF_SHLDRS_EQ"]
period_list = ["QoQ", "YoY", "CAGR_3Y", "CAGR_5Y"]
factor_growth_list = [
    f"{factor}_{periods}"
    for factor, periods in itertools.product(factor_list, period_list)
]

In [ ]:
query = [f"SELECT * FROM `{table}`" for table in factor_list]
query = " UNION ALL ".join(query)

# ------------------------------------------------------
# load from database
# ------------------------------------------------------
with sqlite3.connect(financials_db_path) as conn:
    df_all = (
        pd.read_sql(query, con=conn, parse_dates=["date"])
        .sort_values("date", ignore_index=True)
        .assign(
            variable=lambda x: x["variable"].astype("category"),
            P_SYMBOL=lambda x: x["P_SYMBOL"].astype("category"),
        )
    ).sort_values(["variable", "P_SYMBOL", "date"], ignore_index=True)

display(df_all)
# groupby("variable")を使うことで、df_all全体を何度も走査(loc)するコストをゼロにする
# observed=True はcategory型を使う場合の高速化オプション
grouped = df_all.groupby("variable", observed=True)

# ------------------------------------------------------
# calculate growth and store to database
# ------------------------------------------------------
total_steps = len(factor_list)
for factor_name, df_factor in tqdm(grouped, total=total_steps, desc="Factors"):
    # df_factorはView(参照)の可能性があるため、計算用にコピーを作成
    # ここでメモリを食うが、factor単位なので全体コピーよりは軽い
    # かつ、ループの最後で解放される
    df_base = df_factor.copy()

    for growth in period_list:
        new_variable_name = f"{factor_name}_{growth}"
        df_result = roic_utils.calculate_growth(
            df=df_base, data_name=str(factor_name), growth_type=growth
        )

        # store to database
        db_utils.delete_table_from_database(
            db_path=financials_db_path, table_name=new_variable_name
        )
        factset_utils.store_to_database(
            df=df_result,
            db_path=financials_db_path,
            table_name=new_variable_name,
            verbose=False,
        )

    # ---- メモリ管理 ----
    # 一つのファクター処理が終わったら、使用した一時変数を削除してGC実行
    del df_base
    gc.collect()

,date,P_SYMBOL,value,variable
0,2005-08-31,0HSW-GB,2448.485005,FF_ASSETS
1,2005-09-30,0HSW-GB,2333.443621,FF_ASSETS
2,2005-10-31,0HSW-GB,2333.443621,FF_ASSETS
3,2005-11-30,0HSW-GB,2333.443621,FF_ASSETS
4,2005-12-30,0HSW-GB,NaN,FF_ASSETS
...,...,...,...,...
3165047,2025-06-30,ZURN-CH,24724.999387,FF_SHLDRS_EQ
3165048,2025-07-31,ZURN-CH,24724.999387,FF_SHLDRS_EQ
3165049,2025-08-29,ZURN-CH,24724.999387,FF_SHLDRS_EQ
3165050,2025-09-30,ZURN-CH,24724.999387,FF_SHLDRS_EQ


Factors: 100%|██████████| 4/4 [02:13<00:00, 33.38s/it]


#### 2. ファクターのランクを計算し、データベースに保存する。


In [ ]:
# -----------------------------------
# load data
# -----------------------------------
# 構成銘柄情報
df_weight = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
).assign(date=lambda x: pd.to_datetime(x["date"]))
factset_utils.process_rank_calculation_store_to_db(
    df_weight=df_weight, factor_list=factor_list, financials_db_path=financials_db_path
)

🚀 処理開始: 16 件のタスク


Rank計算進捗: 100%|██████████| 16/16 [02:29<00:00,  9.33s/it]


🎉 全てのランク計算・保存が完了しました


#### 3. データロード&欠損値確認：欠損値はセクター中央値で補完


In [ ]:
# ファクター計算のためのディスクリプターを指定
factor_list = [
    "FF_SALES_CAGR_3Y_PctRank",
    "FF_ASSETS_PctRank",
    "FF_COM_EQ_PctRank",
    "FF_SHLDRS_EQ_PctRank",
]

# -----------------------------------
# load data
# -----------------------------------

df_factor = factset_utils.load_financial_data(
    financials_db_path=financials_db_path, factor_list=factor_list
)

df_weight = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
)
df = (
    factset_utils.merge_idx_constituents_and_financials(
        df_weight=df_weight, df_factor=df_factor
    )
    .fillna(np.nan)
    .dropna(subset=factor_list, how="all")
    .dropna(subset=["Weight (%)"], ignore_index=True)
)
display(df.head())

df = factset_utils.check_missing_value_and_fill_by_sector_median(
    df=df, factor_list=factor_list
)

,date,P_SYMBOL,SEDOL,Asset ID,FG_COMPANY_NAME,GICS Sector,GICS Industry Group,Weight (%),FF_ASSETS_PctRank,FF_COM_EQ_PctRank,FF_SALES_CAGR_3Y_PctRank,FF_SHLDRS_EQ_PctRank
0,2005-08-31,0HSW-GB,3335442,UKIGDP1,Telent PLC,Information Technology,Technology Hardware & Equipment,0.005496,0.421053,0.270677,NaN,0.270677
1,2005-08-31,0II3.XX1-GB,0299303,UKIBKB1,Emap Plc,Consumer Discretionary,Media,0.019501,0.240310,0.128906,NaN,0.128906
2,2005-08-31,0MDJ-GB,0610700,UKIBBF1,Cadbury PLC,Consumer Staples,Food Beverage & Tobacco,0.106288,0.765957,0.621053,NaN,0.621053
3,2005-08-31,0N1N-GB,0230346,UKIBGE1,Arriva Plc Ord,Industrials,Transportation,0.010309,0.331754,0.273585,NaN,0.268868
4,2005-08-31,0N3I-GB,0896265,UKIDBM1,Tomkins PLC,Industrials,Capital Goods,0.020285,0.421801,0.344340,NaN,0.344340


📋 欠損値の状況（補完前）
FF_SALES_CAGR_3Y_PctRank                     : 59,070件 ( 18.5%)
FF_ASSETS_PctRank                            :  1,966件 (  0.6%)
FF_COM_EQ_PctRank                            :  1,756件 (  0.6%)
FF_SHLDRS_EQ_PctRank                         :  1,756件 (  0.6%)

⏳ セクター中央値で補完中...

📋 欠損値の状況（セクター中央値補完後）
⚠️ FF_SALES_CAGR_3Y_PctRank                     : 52,190件 ( 16.4%) | 補完: 6,880件
✅ FF_ASSETS_PctRank                            :      0件 (  0.0%) | 補完: 1,966件
✅ FF_COM_EQ_PctRank                            :      0件 (  0.0%) | 補完: 1,756件
✅ FF_SHLDRS_EQ_PctRank                         :      0件 (  0.0%) | 補完: 1,756件

⚠️  セクター補完で埋まらない欠損値を全体中央値で再補完...
------------------------------------------------------------
⚠️  FF_SALES_CAGR_3Y_PctRank: 全体でも欠損 → 0.5（中立値）で補完
------------------------------------------------------------

✅ 最終欠損値チェック
✅ FF_SALES_CAGR_3Y_PctRank                     : 欠損なし
✅ FF_ASSETS_PctRank                            : 欠損なし
✅ FF_COM_EQ_PctRank                          

#### 4. ファクター計算


In [ ]:
# -----------------------------------
# Factorの計算
# -----------------------------------
# ウェイト設定
blend_weight = {
    "FF_SALES_CAGR_3Y_PctRank": 0.25,
    "FF_ASSETS_PctRank": 0.25,
    "FF_COM_EQ_PctRank": 0.25,
    "FF_SHLDRS_EQ_PctRank": 0.25,
}
factor_name = "Factor_Size"

df = factset_utils.create_factor(
    df=df, factor_name=factor_name, blend_weight=blend_weight
)
display(df.tail(5))

# -----------------------------------
# Store to database
# -----------------------------------
for variable in [f"{factor_name}_Score", f"{factor_name}_Score_Rank"]:
    df_slice = (
        df[["date", "P_SYMBOL", variable]]
        .assign(variable=variable, date=lambda x: pd.to_datetime(x["date"]))
        .rename(columns={variable: "value"})
    )
    db_utils.delete_table_from_database(db_path=financials_db_path, table_name=variable)
    factset_utils.store_to_database(
        df=df_slice,
        db_path=financials_db_path,
        table_name=variable,
        verbose=True,
    )

[autoreload of src.factset_utils failed: Traceback (most recent call last):
  File "c:\Users\Yuki Hata\Desktop\papers\.venv\Lib\site-packages\IPython\extensions\autoreload.py", line 322, in check
    elif self.deduper_reloader.maybe_reload_module(m):
         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^
  File "c:\Users\Yuki Hata\Desktop\papers\.venv\Lib\site-packages\IPython\extensions\deduperreload\deduperreload.py", line 545, in maybe_reload_module
    new_source_code = f.read()
UnicodeDecodeError: 'cp932' codec can't decode byte 0x87 in position 725: illegal multibyte sequence
]



📊 Factor 計算
ウェイト設定:
  FF_SALES_CAGR_3Y_PctRank                     : 25.0%
  FF_ASSETS_PctRank                            : 25.0%
  FF_COM_EQ_PctRank                            : 25.0%
  FF_SHLDRS_EQ_PctRank                         : 25.0%

✅ Score 計算完了
   平均: 0.5039
   標準偏差: 0.2127
   最小値: 0.0150
   最大値: 1.0000

📊 Rank 分布:
  rank5: 63,903件 ( 20.0%)
  rank4: 63,742件 ( 20.0%)
  rank3: 63,744件 ( 20.0%)
  rank2: 63,748件 ( 20.0%)
  rank1: 63,833件 ( 20.0%)

🎯 最終データサンプル:
             date P_SYMBOL    SEDOL             GICS Sector  \
318950 2025-10-31   WTB-GB  B1KJJ40  Consumer Discretionary   
318951 2025-10-31   WTC-AU  BZ8GX83  Information Technology   
318952 2025-10-31  WTRG-US  BLCF3J9               Utilities   
318953 2025-10-31   WTW-US  BDB6Q21              Financials   
318954 2025-10-31    WY-US  2958936             Real Estate   
318955 2025-10-31     X-CA  B8KH5G7              Financials   
318956 2025-10-31   XEL-US  2614807               Utilities   
318957 2025-10-31   XOM-U

,date,P_SYMBOL,SEDOL,Asset ID,FG_COMPANY_NAME,GICS Sector,GICS Industry Group,Weight (%),FF_ASSETS_PctRank,FF_COM_EQ_PctRank,FF_SALES_CAGR_3Y_PctRank,FF_SHLDRS_EQ_PctRank,Factor_Size_Score,Factor_Size_Score_Rank
318965,2025-10-31,ZAL-DE,BQV0SV7,GERZQZ1,ザランド,Consumer Discretionary,Consumer Discretionary Distribution & Retail,0.008099,0.282353,0.364706,0.755814,0.364706,0.441895,rank3
318966,2025-10-31,ZBH-US,2783815,USA4JT1,ジンマー・バイオメット・ホールディングス,Health Care,Health Care Equipment & Services,0.025627,0.534653,0.633663,0.438776,0.633663,0.560189,rank3
318967,2025-10-31,ZBRA-US,2989356,USAP8H1,ゼブラ・テクノロジーズ・コーポレーション Class A,Information Technology,Technology Hardware & Equipment,0.017637,0.349398,0.397590,0.190476,0.397590,0.333764,rank4
318968,2025-10-31,ZTS-US,B95WG16,USBANZ1,ゾエティス Class A,Health Care,Pharmaceuticals Biotechnology & Life Sciences,0.082630,0.415842,0.376238,0.459184,0.376238,0.406875,rank4
318969,2025-10-31,ZURN-CH,5983816,SWIAFM2,チューリッヒ・インシュアランス・グループ,Financials,Insurance,0.131275,0.697436,0.641026,0.952632,0.630769,0.730466,rank1


既存の 0 行との重複をチェックしました。318947 行を新たに追加します。
  -> Factor_Size_Score: データの書き込みが完了しました。
既存の 0 行との重複をチェックしました。318947 行を新たに追加します。
  -> Factor_Size_Score_Rank: データの書き込みが完了しました。


### 4-8. Leverage Factor

-   FF_DEBT_EQ(Debt to Equity ratio)
-   FF_LIABS_SHLDRS_EQ(Total Liabilities to Shareholders' Equity ratio)
-   FF_NET_DEBT(Net Debt)


#### (🚧 実装予定)📝 ディスクリプター追加：NET_DEBT_TO_EBITDA_OPER

-> FF_NET_DEBT / FF_EBITDA_OPER


In [ ]:
descriptor_list = ["FF_NET_DEBT", "FF_EBITDA_OPER"]
# -----------------------------------
# load data
# -----------------------------------
# 構成銘柄情報
df_weight = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
).assign(date=lambda x: pd.to_datetime(x["date"]))
# ファクター
df_factor = factset_utils.load_financial_data(
    financials_db_path=financials_db_path, factor_list=descriptor_list
).assign(date=lambda x: pd.to_datetime(x["date"]))
df_merged = factset_utils.merge_idx_constituents_and_financials(
    df_weight=df_weight, df_factor=df_factor
)

df_merged["NET_DEBT_TO_EBITDA_OPER"] = df_merged["FF_NET_DEBT"].div(
    df_merged["FF_EBITDA_OPER"]
)
display(df_merged.loc[df_merged["NET_DEBT_TO_EBITDA_OPER"] < 0])

,date,P_SYMBOL,SEDOL,Asset ID,FG_COMPANY_NAME,GICS Sector,GICS Industry Group,Weight (%),FF_EBITDA_OPER,FF_NET_DEBT,NET_DEBT_TO_EBITDA_OPER
78939,2005-08-31,0UAN-GB,0128269,UKIAYK1,Invesco Ltd.,Financials,Financial Services,0.025020,156.54217,-320.523897,-2.047524
78956,2005-08-31,2878-HK,B00G568,HKGCXQ1,Solomon Systech (International) Ltd.,Information Technology,Semiconductors & Semiconductor Equipment,0.002918,45.57000,-152.981004,-3.357055
78960,2005-08-31,315-HK,6856995,HKGBCL1,SmarTone Telecommunications Holdings Limited,Communication Services,Telecommunication Services,0.001761,50.32445,-148.727957,-2.955382
78963,2005-08-31,330-HK,6321642,HKGAXW1,Esprit Holdings Limited,Consumer Discretionary,Consumer Discretionary Distribution & Retail,0.036683,283.80326,-222.383163,-0.783582
78966,2005-08-31,494-HK,6286257,HKGAJG1,Li & Fung Limited,Consumer Discretionary,Consumer Discretionary Distribution & Retail,0.018461,76.74025,-18.533442,-0.241509
...,...,...,...,...,...,...,...,...,...,...,...
400839,2025-10-31,WST-US,2950482,USAOV41,ウエスト・ファーマシューティカル・サービシーズ,Health Care,Pharmaceuticals Biotechnology & Life Sciences,0.026103,216.70000,-325.500000,-1.502077
400841,2025-10-31,WTC-AU,BZ8GX83,AUSIDZ1,ワイズテック・グローバル,Information Technology,Software & Services,0.011702,218.60000,-55.700003,-0.254803
400848,2025-10-31,XRO-AU,B8P4LP4,AUSHTH1,ゼロ,Information Technology,Software & Services,0.020128,208.45848,-2004.614013,-9.616371
400850,2025-10-31,XYZ-US,BYNZGK1,USBDBN1,"Block, Inc. Class A",Financials,Financial Services,0.054288,865.35900,-1336.948000,-1.544963


#### 1. ファクター値を計算してデータベースに保存する


In [ ]:
factor_list = ["FF_DEBT_EQ", "FF_LIABS_SHLDRS_EQ", "FF_NET_DEBT"]

In [ ]:
# -----------------------------------
# load data
# -----------------------------------
# 構成銘柄情報
df_weight = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
).assign(date=lambda x: pd.to_datetime(x["date"]))

factset_utils.process_rank_calculation_store_to_db(
    df_weight=df_weight, factor_list=factor_list, financials_db_path=financials_db_path
)

[autoreload of src.factset_utils failed: Traceback (most recent call last):
  File "c:\Users\Yuki Hata\Desktop\papers\.venv\Lib\site-packages\IPython\extensions\autoreload.py", line 322, in check
    elif self.deduper_reloader.maybe_reload_module(m):
         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^
  File "c:\Users\Yuki Hata\Desktop\papers\.venv\Lib\site-packages\IPython\extensions\deduperreload\deduperreload.py", line 545, in maybe_reload_module
    new_source_code = f.read()
UnicodeDecodeError: 'cp932' codec can't decode byte 0x87 in position 725: illegal multibyte sequence
]


🚀 処理開始: 3 件のタスク


Rank計算進捗: 100%|██████████| 3/3 [00:32<00:00, 10.69s/it]


🎉 全てのランク計算・保存が完了しました


データロード&欠損値確認: 欠損値はセクター中央値で補完


In [ ]:
# ファクター計算のためのディスクリプターを指定
factor_list = [
    "FF_DEBT_EQ_Inv_PctRank",
    "FF_LIABS_SHLDRS_EQ_Inv_PctRank",
    "FF_NET_DEBT_Inv_PctRank",
]

# -----------------------------------
# load data
# -----------------------------------

df_factor = factset_utils.load_financial_data(
    financials_db_path=financials_db_path, factor_list=factor_list
)

df_weight = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
)
df = (
    factset_utils.merge_idx_constituents_and_financials(
        df_weight=df_weight, df_factor=df_factor
    )
    .fillna(np.nan)
    .dropna(subset=factor_list, how="all")
    .dropna(subset=["Weight (%)"], ignore_index=True)
)
display(df.head())

df = factset_utils.check_missing_value_and_fill_by_sector_median(
    df=df, factor_list=factor_list
)

,date,P_SYMBOL,SEDOL,Asset ID,FG_COMPANY_NAME,GICS Sector,GICS Industry Group,Weight (%),FF_DEBT_EQ_Inv_PctRank,FF_LIABS_SHLDRS_EQ_Inv_PctRank,FF_NET_DEBT_Inv_PctRank
0,2005-08-31,0HSW-GB,3335442,UKIGDP1,Telent PLC,Information Technology,Technology Hardware & Equipment,0.005496,0.542636,0.578947,NaN
1,2005-08-31,0II3.XX1-GB,0299303,UKIBKB1,Emap Plc,Consumer Discretionary,Media,0.019501,0.197531,0.758755,NaN
2,2005-08-31,0MDJ-GB,0610700,UKIBBF1,Cadbury PLC,Consumer Staples,Food Beverage & Tobacco,0.106288,0.108696,0.234043,0.129870
3,2005-08-31,0N1N-GB,0230346,UKIBGE1,Arriva Plc Ord,Industrials,Transportation,0.010309,0.323671,0.671429,0.564246
4,2005-08-31,0N3I-GB,0896265,UKIDBM1,Tomkins PLC,Industrials,Capital Goods,0.020285,0.328502,0.580952,0.519553


📋 欠損値の状況（補完前）
FF_DEBT_EQ_Inv_PctRank                       :  7,872件 (  2.5%)
FF_LIABS_SHLDRS_EQ_Inv_PctRank               :    300件 (  0.1%)
FF_NET_DEBT_Inv_PctRank                      : 18,313件 (  5.8%)

⏳ セクター中央値で補完中...

📋 欠損値の状況（セクター中央値補完後）
✅ FF_DEBT_EQ_Inv_PctRank                       :      0件 (  0.0%) | 補完: 7,872件
✅ FF_LIABS_SHLDRS_EQ_Inv_PctRank               :      0件 (  0.0%) | 補完: 300件
✅ FF_NET_DEBT_Inv_PctRank                      :      0件 (  0.0%) | 補完: 18,313件

✅ 最終欠損値チェック
✅ FF_DEBT_EQ_Inv_PctRank                       : 欠損なし
✅ FF_LIABS_SHLDRS_EQ_Inv_PctRank               : 欠損なし
✅ FF_NET_DEBT_Inv_PctRank                      : 欠損なし
🎉 すべての欠損値が補完されました！


ファクター計算


In [ ]:
# -----------------------------------
# Factorの計算
# -----------------------------------
# ウェイト設定
blend_weight = {
    "FF_DEBT_EQ_Inv_PctRank": 1 / 3,
    "FF_LIABS_SHLDRS_EQ_Inv_PctRank": 1 / 3,
    "FF_NET_DEBT_Inv_PctRank": 1 / 3,
}
factor_name = "Factor_Leverage"

df = factset_utils.create_factor(
    df=df, factor_name=factor_name, blend_weight=blend_weight
)
display(df.tail(5))

# -----------------------------------
# Store to database
# -----------------------------------
for variable in [f"{factor_name}_Score", f"{factor_name}_Score_Rank"]:
    df_slice = (
        df[["date", "P_SYMBOL", variable]]
        .assign(variable=variable, date=lambda x: pd.to_datetime(x["date"]))
        .rename(columns={variable: "value"})
    )
    db_utils.delete_table_from_database(db_path=financials_db_path, table_name=variable)
    factset_utils.store_to_database(
        df=df_slice,
        db_path=financials_db_path,
        table_name=variable,
        verbose=True,
    )


📊 Factor 計算
ウェイト設定:
  FF_DEBT_EQ_Inv_PctRank                       : 33.3%
  FF_LIABS_SHLDRS_EQ_Inv_PctRank               : 33.3%
  FF_NET_DEBT_Inv_PctRank                      : 33.3%

✅ Score 計算完了
   平均: 0.4959
   標準偏差: 0.2305
   最小値: 0.0000
   最大値: 0.9831

📊 Rank 分布:
  rank5: 63,534件 ( 20.0%)
  rank4: 63,382件 ( 20.0%)
  rank3: 63,389件 ( 20.0%)
  rank2: 63,377件 ( 20.0%)
  rank1: 63,478件 ( 20.0%)

🎯 最終データサンプル:
             date P_SYMBOL    SEDOL             GICS Sector  \
317140 2025-10-31   WTB-GB  B1KJJ40  Consumer Discretionary   
317141 2025-10-31   WTC-AU  BZ8GX83  Information Technology   
317142 2025-10-31  WTRG-US  BLCF3J9               Utilities   
317143 2025-10-31   WTW-US  BDB6Q21              Financials   
317144 2025-10-31    WY-US  2958936             Real Estate   
317145 2025-10-31     X-CA  B8KH5G7              Financials   
317146 2025-10-31   XEL-US  2614807               Utilities   
317147 2025-10-31   XOM-US  2326618                  Energy   
317148 2025-10-31

,date,P_SYMBOL,SEDOL,Asset ID,FG_COMPANY_NAME,GICS Sector,GICS Industry Group,Weight (%),FF_DEBT_EQ_Inv_PctRank,FF_LIABS_SHLDRS_EQ_Inv_PctRank,FF_NET_DEBT_Inv_PctRank,Factor_Leverage_Score,Factor_Leverage_Score_Rank
317155,2025-10-31,ZAL-DE,BQV0SV7,GERZQZ1,ザランド,Consumer Discretionary,Consumer Discretionary Distribution & Retail,0.008099,0.689189,0.717647,0.788235,0.731691,rank1
317156,2025-10-31,ZBH-US,2783815,USA4JT1,ジンマー・バイオメット・ホールディングス,Health Care,Health Care Equipment & Services,0.025627,0.536842,0.465347,0.396040,0.466076,rank3
317157,2025-10-31,ZBRA-US,2989356,USAP8H1,ゼブラ・テクノロジーズ・コーポレーション Class A,Information Technology,Technology Hardware & Equipment,0.017637,0.362500,0.650602,0.421687,0.478263,rank3
317158,2025-10-31,ZTS-US,B95WG16,USBANZ1,ゾエティス Class A,Health Care,Pharmaceuticals Biotechnology & Life Sciences,0.082630,0.136842,0.584158,0.465347,0.395449,rank4
317159,2025-10-31,ZURN-CH,5983816,SWIAFM2,チューリッヒ・インシュアランス・グループ,Financials,Insurance,0.131275,0.583333,0.304124,0.371134,0.419530,rank4


既存の 0 行との重複をチェックしました。317137 行を新たに追加します。
  -> Factor_Leverage_Score: データの書き込みが完了しました。
既存の 0 行との重複をチェックしました。317137 行を新たに追加します。
  -> Factor_Leverage_Score_Rank: データの書き込みが完了しました。


### 4-9. Momentum/Reversal Factor


#### 1. 区間リターンを計算


In [ ]:
# ----------------------------------------
# load price data
# ----------------------------------------
df_price = (
    roic_utils.load_FG_PRICE(db_path=financials_db_path)
    .reset_index()
    .astype({"P_SYMBOL": "category"})
    .assign(date=lambda x: pd.to_datetime(x["date"]))
).sort_values(["date", "P_SYMBOL"], ignore_index=True)

# ----------------------------------------
# 区間リターンを計算
# ----------------------------------------

df_calculated = roic_utils.calculate_interval_returns(df=df_price).drop(
    columns=["FG_PRICE"]
)
target_columns = [c for c in df_calculated.columns if c not in ["date", "P_SYMBOL"]]

# store to database
for col_name in target_columns:
    df_slice = df_calculated[["date", "P_SYMBOL", col_name]].copy()
    df_slice = (
        df_slice.rename(columns={col_name: "value"})
        .assign(variable=col_name)
        .reindex(columns=["date", "P_SYMBOL", "variable", "value"])
    )
    df_slice = df_slice.dropna(subset=["value"])
    factset_utils.store_to_database(
        df=df_slice, db_path=financials_db_path, table_name=col_name
    )

既存の 0 行との重複をチェックしました。354363 行を新たに追加します。
  -> Return_1YAgo_to_Current: データの書き込みが完了しました。
既存の 0 行との重複をチェックしました。355091 行を新たに追加します。
  -> Return_1YAgo_to_1MAgo: データの書き込みが完了しました。
既存の 0 行との重複をチェックしました。338974 行を新たに追加します。
  -> Return_2YAgo_to_1YAgo: データの書き込みが完了しました。
既存の 0 行との重複をチェックしました。318456 行を新たに追加します。
  -> Return_3YAgo_to_2YAgo: データの書き込みが完了しました。
既存の 0 行との重複をチェックしました。310497 行を新たに追加します。
  -> Return_3YAgo_to_1YAgo: データの書き込みが完了しました。
既存の 0 行との重複をチェックしました。381041 行を新たに追加します。
  -> Return_1MAgo_to_Current: データの書き込みが完了しました。


#### 2. ファクター値を計算して DB 保存


In [ ]:
# inverse = False
momentum_descriptor_list = [
    "Return_1YAgo_to_Current",
    "Return_1YAgo_to_1MAgo",
    "Return_2YAgo_to_1YAgo",
    "Return_3YAgo_to_2YAgo",
]

# inverse = True
reversal_descriptor_list = [
    "Return_3YAgo_to_1YAgo",
    "Return_1MAgo_to_Current",
]

In [ ]:
# -----------------------------------
# load data
# -----------------------------------
# 構成銘柄情報
df_weight = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
).assign(date=lambda x: pd.to_datetime(x["date"]))

# -----------------------------------
# Momentum -> inversed=False
# -----------------------------------
# セクター中立なし
# process_rank_calculation_store_to_db(
#     df_weight=df_weight,
#     factor_list=momentum_descriptor_list,
#     financials_db_path=financials_db_path,
#     sector_neutral_mode=False,
#     inversed=False,
# )

# # セクター中立あり
# process_rank_calculation_store_to_db(
#     df_weight=df_weight,
#     factor_list=momentum_descriptor_list,
#     financials_db_path=financials_db_path,
#     sector_neutral_mode=True,
#     inversed=False,
# )

# -----------------------------------
# Reversal -> inversed=True
# -----------------------------------
# セクター中立なし
factset_utils.process_rank_calculation_store_to_db(
    df_weight=df_weight,
    factor_list=reversal_descriptor_list,
    financials_db_path=financials_db_path,
    sector_neutral_mode=False,
    inversed=True,
)

# セクター中立あり
factset_utils.process_rank_calculation_store_to_db(
    df_weight=df_weight,
    factor_list=reversal_descriptor_list,
    financials_db_path=financials_db_path,
    sector_neutral_mode=True,
    inversed=True,
)

🚀 処理開始: 2 件のタスク (Single Factor Mode)


Rank計算進捗: 100%|██████████| 2/2 [00:15<00:00,  7.53s/it]


🎉 全てのランク計算・保存が完了しました
🚀 処理開始: 2 件のタスク (Single Factor Mode)


Rank計算進捗: 100%|██████████| 2/2 [00:17<00:00,  8.76s/it]


🎉 全てのランク計算・保存が完了しました


#### 3. データロード&欠損値確認: 欠損値はセクター中央値で補完


In [ ]:
# ファクター計算のためのディスクリプターを指定
factor_list = [s + "_PctRank" for s in momentum_descriptor_list]

# -----------------------------------
# load data
# -----------------------------------

df_factor = factset_utils.load_financial_data(
    financials_db_path=financials_db_path, factor_list=factor_list
)

df_weight = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
)
df = (
    factset_utils.merge_idx_constituents_and_financials(
        df_weight=df_weight, df_factor=df_factor
    )
    .fillna(np.nan)
    .dropna(subset=factor_list, how="all")
    .dropna(subset=["Weight (%)"], ignore_index=True)
)
df = df.loc[df["date"].dt.year >= 2009]
display(df.head())

df = factset_utils.check_missing_value_and_fill_by_sector_median(
    df=df, factor_list=factor_list
)

,date,P_SYMBOL,SEDOL,Asset ID,FG_COMPANY_NAME,GICS Sector,GICS Industry Group,Weight (%),Return_1YAgo_to_1MAgo_PctRank,Return_1YAgo_to_Current_PctRank,Return_2YAgo_to_1YAgo_PctRank,Return_3YAgo_to_2YAgo_PctRank
30834,2009-03-31,0MDJ-GB,B2PF6M7,UKIBBF1,Cadbury PLC,Consumer Staples,Food Beverage & Tobacco,0.078896,0.737864,0.747573,0.247525,0.356436
30835,2009-03-31,0N3I-GB,0896265,UKIDBM1,Tomkins PLC,Industrials,Capital Goods,0.011803,0.722772,0.742574,0.071066,0.031250
30836,2009-03-31,0P7J-GB,0028262,UKIAPY1,Amec Foster Wheeler plc,Energy,Energy,0.019520,0.854545,0.790909,0.688073,0.924528
30837,2009-03-31,1-HK,6190273,HKGAAE1,CKハチソン・ホールディングス,Financials,Real Estate,0.091940,0.711191,0.689531,0.923358,0.687500
30838,2009-03-31,10-HK,6408352,HKGAGG1,Hang Lung Group Limited,Financials,Real Estate,0.020283,0.696751,0.736462,0.985401,0.959559


📋 欠損値の状況（補完前）
Return_1YAgo_to_Current_PctRank              :      6件 (  0.0%)
Return_1YAgo_to_1MAgo_PctRank                :      0件 (  0.0%)
Return_2YAgo_to_1YAgo_PctRank                :  2,146件 (  1.2%)
Return_3YAgo_to_2YAgo_PctRank                :  4,703件 (  2.6%)

⏳ セクター中央値で補完中...

📋 欠損値の状況（セクター中央値補完後）
✅ Return_1YAgo_to_Current_PctRank              :      0件 (  0.0%) | 補完: 6件
✅ Return_1YAgo_to_1MAgo_PctRank                :      0件 (  0.0%) | 補完: 0件
✅ Return_2YAgo_to_1YAgo_PctRank                :      0件 (  0.0%) | 補完: 2,146件
✅ Return_3YAgo_to_2YAgo_PctRank                :      0件 (  0.0%) | 補完: 4,703件

✅ 最終欠損値チェック
✅ Return_1YAgo_to_Current_PctRank              : 欠損なし
✅ Return_1YAgo_to_1MAgo_PctRank                : 欠損なし
✅ Return_2YAgo_to_1YAgo_PctRank                : 欠損なし
✅ Return_3YAgo_to_2YAgo_PctRank                : 欠損なし
🎉 すべての欠損値が補完されました！


#### （🚧 工事中）4. ファクター計算


### 4-10. その他の財務項目

時点でのランク、パーセントランク、ZScore のみ計算


In [ ]:
# ファクター値
factor_list = [
    "FF_ASSETS",
    "FF_BPS",
    "FF_BPS_TANG",
    "FF_CAPEX",
    "FF_CASH_ST",
    "FF_COGS",
    "FF_COM_EQ",
    "FF_CURR_RATIO",
    "FF_DEBT",
    "FF_DEBT_ENTRPR_VAL",
    "FF_DEBT_EQ",
    "FF_DEBT_LT",
    "FF_DEBT_ST",
    "FF_DEP_AMORT_EXP",
    "FF_DIV_YLD",
    "FF_DPS",
    "FF_EBITDA_OPER",
    "FF_EBITDA_OPER_MGN",
    "FF_EBIT_OPER",
    "FF_EBIT_OPER_MGN",
    "FF_ENTRPR_VAL_EBITDA_OPER",
    "FF_ENTRPR_VAL_EBIT_OPER",
    "FF_ENTRPR_VAL_SALES",
    "FF_EPS",
    "FF_EPS_DIL",
    "FF_FREE_CF",
    "FF_FREE_PS_CF",
    "FF_GROSS_INC",
    "FF_GROSS_MGN",
    "FF_INC_TAX",
    "FF_INT_EXP_NET",
    "FF_LIABS",
    "FF_LIABS_SHLDRS_EQ",
    "FF_MIN_INT_ACCUM",
    "FF_NET_DEBT",
    "FF_NET_INC",
    "FF_NET_MGN",
    "FF_OPER_CF",
    "FF_OPER_INC",
    "FF_OPER_MGN",
    "FF_OPER_PS_NET_CF",
    "FF_PAY_OUT_RATIO",
    "FF_PBK",
    "FF_PE",
    "FF_PFD_STK",
    "FF_PPE_NET",
    "FF_PSALES",
    "FF_PTX_INC",
    "FF_PTX_MGN",
    "FF_QUICK_RATIO",
    "FF_ROA",
    "FF_ROE",
    "FF_ROIC",
    "FF_ROTC",
    "FF_SALES",
    "FF_SALES_PS",
    "FF_SGA",
    "FF_SHLDRS_EQ",
    "FF_STK_OPT_EXP",
    "FF_STK_PURCH_CF",
    "FF_TAX_RATE",
    "FF_WKCAP",
]

In [ ]:
# -----------------------------------
# load data
# -----------------------------------
# 構成銘柄情報
df_weight = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
).assign(date=lambda x: pd.to_datetime(x["date"]))

factset_utils.process_rank_calculation_store_to_db(
    df_weight=df_weight, factor_list=factor_list, financials_db_path=financials_db_path
)

Rank計算進捗: 100%|██████████| 62/62 [08:56<00:00,  8.65s/it]


🎉 全てのランク計算・保存が完了しました


### ✅ データベース内容確認


In [ ]:
table_names = sorted(db_utils.get_table_names(db_path=financials_db_path))
print(f"全{len(table_names)}テーブル")
display(table_names)

with sqlite3.connect(factset_index_db_path) as conn:
    df = pd.read_sql(
        f"SELECT * FROM {UNIVERSE_CODE} LIMIT 5", parse_dates=["date"], con=conn
    )
    display(df)
    display(df.columns)

全714テーブル


['Active_Return_12M',
 'Active_Return_12M_annlzd',
 'Active_Return_1M',
 'Active_Return_1M_annlzd',
 'Active_Return_3M',
 'Active_Return_3M_annlzd',
 'Active_Return_3Y',
 'Active_Return_3Y_annlzd',
 'Active_Return_5Y',
 'Active_Return_5Y_annlzd',
 'Active_Return_6M',
 'Active_Return_6M_annlzd',
 'FF_ASSETS',
 'FF_ASSETS_CAGR_3Y',
 'FF_ASSETS_CAGR_3Y_PctRank',
 'FF_ASSETS_CAGR_3Y_Rank',
 'FF_ASSETS_CAGR_3Y_ZScore',
 'FF_ASSETS_CAGR_5Y',
 'FF_ASSETS_CAGR_5Y_PctRank',
 'FF_ASSETS_CAGR_5Y_Rank',
 'FF_ASSETS_CAGR_5Y_ZScore',
 'FF_ASSETS_PctRank',
 'FF_ASSETS_QoQ',
 'FF_ASSETS_QoQ_PctRank',
 'FF_ASSETS_QoQ_Rank',
 'FF_ASSETS_QoQ_ZScore',
 'FF_ASSETS_Rank',
 'FF_ASSETS_YoY',
 'FF_ASSETS_YoY_PctRank',
 'FF_ASSETS_YoY_Rank',
 'FF_ASSETS_YoY_ZScore',
 'FF_ASSETS_ZScore',
 'FF_BPS',
 'FF_BPS_PctRank',
 'FF_BPS_Rank',
 'FF_BPS_TANG',
 'FF_BPS_TANG_PctRank',
 'FF_BPS_TANG_Rank',
 'FF_BPS_TANG_ZScore',
 'FF_BPS_ZScore',
 'FF_CAPEX',
 'FF_CAPEX_CAGR_3Y',
 'FF_CAPEX_CAGR_3Y_PctRank',
 'FF_CAPEX_CAGR_3

,Universe,Universe_code_BPM,date,Name,Bloomberg Ticker,BloombergID,Asset ID,Asset ID Type,SEDOL,Country,...,FG_COMPANY_NAME_CUSIP,P_SYMBOL_CUSIP,ISIN,FG_COMPANY_NAME_ISIN,P_SYMBOL_ISIN,CODE_JP,FG_COMPANY_NAME_CODE_JP,P_SYMBOL_CODE_JP,P_SYMBOL,FG_COMPANY_NAME
0,MSCI KOKUSAI - Daily,MSXJPN_AD,2000-01-31,21ST CENTURY FOX,None,None,AUSBIN2,BARRAID,662075,AUS,...,None,None,None,None,None,None,None,None,FOXLV-AU,Twenty-First Century Fox Inc. Class A CDI
1,MSCI KOKUSAI - Daily,MSXJPN_AD,2000-01-31,21ST CENTURY FOX,None,None,AUSBIN1,BARRAID,688692,AUS,...,None,None,None,None,None,None,None,None,FOX-AU,Twenty-First Century Fox Inc. Class B CDI
2,MSCI KOKUSAI - Daily,MSXJPN_AD,2000-01-31,3I GROUP PLC,None,None,UKIENL1,BARRAID,0888693,GBR,...,スリーアイ・グループ,TGOPF-US,GB0008886938,スリーアイ・グループ,III-GB,None,None,None,III-GB,スリーアイ・グループ
3,MSCI KOKUSAI - Daily,MSXJPN_AD,2000-01-31,3M CO,None,None,USAJ8P1,BARRAID,2595708,USA,...,3Mカンパニー,MMM-US,US6040591058,3Mカンパニー,MMM-US,None,None,None,MMM-US,3Mカンパニー
4,MSCI KOKUSAI - Daily,MSXJPN_AD,2000-01-31,ABB LTD,None,None,SWIAAN1,BARRAID,5661190,CHE,...,ABB,None,CH0003846620,ABB,ABBN-CH,None,None,None,ABBN-CH,ABB


Index(['Universe', 'Universe_code_BPM', 'date', 'Name', 'Bloomberg Ticker',
       'BloombergID', 'Asset ID', 'Asset ID Type', 'SEDOL', 'Country',
       'GICS Sector', 'GICS Industry', 'GICS Industry Group',
       'GICS Sub-Industry', 'Holdings', 'Weight (%)', 'Mkt Value',
       'FG_COMPANY_NAME_SEDOL', 'P_SYMBOL_SEDOL', 'CUSIP',
       'FG_COMPANY_NAME_CUSIP', 'P_SYMBOL_CUSIP', 'ISIN',
       'FG_COMPANY_NAME_ISIN', 'P_SYMBOL_ISIN', 'CODE_JP',
       'FG_COMPANY_NAME_CODE_JP', 'P_SYMBOL_CODE_JP', 'P_SYMBOL',
       'FG_COMPANY_NAME'],
      dtype='object')

## 5. 欠損確認


### ✅ データベース確認


In [ ]:
table_names = sorted(db_utils.get_table_names(db_path=financials_db_path))
print(f"全{len(table_names)}テーブル")
display(table_names)

with sqlite3.connect(factset_index_db_path) as conn:
    df_weight = pd.read_sql(
        f"SELECT * FROM {UNIVERSE_CODE}", parse_dates=["date"], con=conn
    )
    display(df_weight.columns)
    display(df_weight.tail(5))

全454テーブル


['Active_Return_1M',
 'Active_Return_1M_annlzd',
 'Active_Return_3M',
 'Active_Return_3M_annlzd',
 'Active_Return_3Y',
 'Active_Return_3Y_annlzd',
 'Active_Return_5Y',
 'Active_Return_5Y_annlzd',
 'Active_Return_6M',
 'Active_Return_6M_annlzd',
 'FF_ASSETS',
 'FF_ASSETS_CAGR_3Y',
 'FF_ASSETS_CAGR_3Y_PctRank',
 'FF_ASSETS_CAGR_3Y_Rank',
 'FF_ASSETS_CAGR_3Y_ZScore',
 'FF_ASSETS_CAGR_5Y',
 'FF_ASSETS_CAGR_5Y_PctRank',
 'FF_ASSETS_CAGR_5Y_Rank',
 'FF_ASSETS_CAGR_5Y_ZScore',
 'FF_ASSETS_PctRank',
 'FF_ASSETS_QoQ',
 'FF_ASSETS_QoQ_PctRank',
 'FF_ASSETS_QoQ_Rank',
 'FF_ASSETS_QoQ_ZScore',
 'FF_ASSETS_Rank',
 'FF_ASSETS_YoY',
 'FF_ASSETS_YoY_PctRank',
 'FF_ASSETS_YoY_Rank',
 'FF_ASSETS_YoY_ZScore',
 'FF_ASSETS_ZScore',
 'FF_BPS',
 'FF_BPS_PctRank',
 'FF_BPS_Rank',
 'FF_BPS_TANG',
 'FF_BPS_TANG_PctRank',
 'FF_BPS_TANG_Rank',
 'FF_BPS_TANG_ZScore',
 'FF_BPS_ZScore',
 'FF_CAPEX',
 'FF_CAPEX_PctRank',
 'FF_CAPEX_Rank',
 'FF_CAPEX_ZScore',
 'FF_CASH_ST',
 'FF_CASH_ST_PctRank',
 'FF_CASH_ST_Rank',
 

Index(['Universe', 'Universe_code_BPM', 'date', 'Name', 'Bloomberg Ticker',
       'BloombergID', 'Asset ID', 'Asset ID Type', 'SEDOL', 'Country',
       'GICS Sector', 'GICS Industry', 'GICS Industry Group',
       'GICS Sub-Industry', 'Holdings', 'Weight (%)', 'Mkt Value',
       'FG_COMPANY_NAME_SEDOL', 'P_SYMBOL_SEDOL', 'CUSIP',
       'FG_COMPANY_NAME_CUSIP', 'P_SYMBOL_CUSIP', 'ISIN',
       'FG_COMPANY_NAME_ISIN', 'P_SYMBOL_ISIN', 'CODE_JP',
       'FG_COMPANY_NAME_CODE_JP', 'P_SYMBOL_CODE_JP', 'P_SYMBOL',
       'FG_COMPANY_NAME'],
      dtype='object')

,Universe,Universe_code_BPM,date,Name,Bloomberg Ticker,BloombergID,Asset ID,Asset ID Type,SEDOL,Country,...,FG_COMPANY_NAME_CUSIP,P_SYMBOL_CUSIP,ISIN,FG_COMPANY_NAME_ISIN,P_SYMBOL_ISIN,CODE_JP,FG_COMPANY_NAME_CODE_JP,P_SYMBOL_CODE_JP,P_SYMBOL,FG_COMPANY_NAME
400859,MSCI KOKUSAI - Daily,MSXJPN_AD,2025-10-31,ZIMMER BIOMET HOLDINGS INC,None,None,USA4JT1,BARRAID,2783815,USA,...,ジンマー・バイオメット・ホールディングス,ZBH-US,US98956P1021,ジンマー・バイオメット・ホールディングス,ZBH-US,None,None,None,ZBH-US,ジンマー・バイオメット・ホールディングス
400860,MSCI KOKUSAI - Daily,MSXJPN_AD,2025-10-31,ZOETIS INC,None,None,USBANZ1,BARRAID,B95WG16,USA,...,ゾエティス Class A,ZTS-US,US98978V1035,ゾエティス Class A,ZTS-US,None,None,None,ZTS-US,ゾエティス Class A
400861,MSCI KOKUSAI - Daily,MSXJPN_AD,2025-10-31,ZOOM COMMUNICATIONS INC,None,None,USBEOV1,BARRAID,BGSP7M9,USA,...,ズーム・ビデオ・コミュニケーションズ Class A,ZM-US,US98980L1017,ズーム・ビデオ・コミュニケーションズ Class A,ZM-US,None,None,None,ZM-US,ズーム・ビデオ・コミュニケーションズ Class A
400862,MSCI KOKUSAI - Daily,MSXJPN_AD,2025-10-31,ZSCALER INC,None,None,USBDYI1,BARRAID,BZ00V34,USA,...,ゼットスケイラー,ZS-US,US98980G1022,ゼットスケイラー,ZS-US,None,None,None,ZS-US,ゼットスケイラー
400863,MSCI KOKUSAI - Daily,MSXJPN_AD,2025-10-31,ZURICH INSURANCE GROUP AG,None,None,SWIAFM2,BARRAID,5983816,CHE,...,チューリッヒ・インシュアランス・グループ,ZFSVF-US,CH0011075394,チューリッヒ・インシュアランス・グループ,ZURN-CH,None,None,None,ZURN-CH,チューリッヒ・インシュアランス・グループ


In [ ]:
# 各変数を処理する関数
def process_variable(variable, financials_db_path, df_weight):
    with sqlite3.connect(financials_db_path) as conn:
        df_factor = pd.read_sql(
            f"SELECT `date`, `P_SYMBOL`, `value` FROM `{variable}`",
            con=conn,
            parse_dates=["date"],
        )

    merged_df = (
        pd.merge(df_weight, df_factor, on=["date", "P_SYMBOL"], how="outer")
        .rename(columns={"value": variable})
        .dropna(subset=["Weight (%)", variable], how="any", axis=0)
    ).fillna(np.nan)

    g = (
        pd.DataFrame(merged_df.groupby(["date"])["Weight (%)"].agg("sum"))
        .reset_index()
        .assign(variable=variable)
    )

    return g

In [ ]:
dfs_weight_sum = []
with ThreadPoolExecutor(max_workers=4) as executor:
    future_to_var = {
        executor.submit(process_variable, var, financials_db_path, df_weight): var
        for var in table_names
    }

    for future in as_completed(future_to_var):
        variable = future_to_var[future]
        try:
            result = future.result()
            if result is not None:
                dfs_weight_sum.append(result)
                print(f"✓ Completed: {variable}")
        except Exception as e:
            print(f"✗ Failed {variable}: {e}")

✓ Completed: Active_Return_3M
✓ Completed: Active_Return_1M_annlzd
✓ Completed: Active_Return_1M
✓ Completed: Active_Return_3M_annlzd
✓ Completed: Active_Return_3Y
✓ Completed: Active_Return_3Y_annlzd
✓ Completed: Active_Return_5Y
✓ Completed: Active_Return_5Y_annlzd
✓ Completed: Active_Return_6M
✓ Completed: FF_ASSETS
✓ Completed: Active_Return_6M_annlzd
✓ Completed: FF_ASSETS_CAGR_3Y
✓ Completed: FF_ASSETS_CAGR_5Y
✓ Completed: FF_ASSETS_CAGR_3Y_Rank
✓ Completed: FF_ASSETS_CAGR_3Y_PctRank
✓ Completed: FF_ASSETS_CAGR_3Y_ZScore
✓ Completed: FF_ASSETS_CAGR_5Y_PctRank
✓ Completed: FF_ASSETS_CAGR_5Y_Rank
✓ Completed: FF_ASSETS_CAGR_5Y_ZScore
✓ Completed: FF_ASSETS_PctRank
✓ Completed: FF_ASSETS_QoQ
✓ Completed: FF_ASSETS_QoQ_Rank
✓ Completed: FF_ASSETS_QoQ_PctRank
✓ Completed: FF_ASSETS_QoQ_ZScore
✓ Completed: FF_ASSETS_Rank
✓ Completed: FF_ASSETS_YoY
✓ Completed: FF_ASSETS_YoY_PctRank
✓ Completed: FF_ASSETS_YoY_Rank
✓ Completed: FF_ASSETS_YoY_ZScore
✓ Completed: FF_BPS
✓ Completed: FF_ASS

In [ ]:
df_weight_sum = pd.concat(dfs_weight_sum).sort_values(
    ["date", "Weight (%)"], ignore_index=True
)
df_weight_sum = (
    pd.pivot(df_weight_sum, index=["date"], columns="variable", values="Weight (%)")
    .reset_index()
    .filter(regex="date|_Rank|_PctRank|_ZScore")
)
display(df_weight_sum)

output_path = BLOOMBERG_DATA_DIR / f"{UNIVERSE_CODE}_not_missing_weight.xlsx"
df_weight_sum.to_excel(output_path, index=False)

variable,date,FF_ASSETS_CAGR_3Y_PctRank,FF_ASSETS_CAGR_3Y_Rank,FF_ASSETS_CAGR_3Y_ZScore,FF_ASSETS_CAGR_5Y_PctRank,FF_ASSETS_CAGR_5Y_Rank,FF_ASSETS_CAGR_5Y_ZScore,FF_ASSETS_PctRank,FF_ASSETS_QoQ_PctRank,FF_ASSETS_QoQ_Rank,...,FF_STK_OPT_EXP_ZScore,FF_STK_PURCH_CF_PctRank,FF_STK_PURCH_CF_Rank,FF_STK_PURCH_CF_ZScore,FF_TAX_RATE_PctRank,FF_TAX_RATE_Rank,FF_TAX_RATE_ZScore,FF_WKCAP_PctRank,FF_WKCAP_Rank,FF_WKCAP_ZScore
0,2005-08-31,NaN,NaN,NaN,NaN,NaN,NaN,97.532947,NaN,NaN,...,15.983133,61.725108,61.725108,59.932037,93.525348,93.525348,93.525348,89.346444,89.346444,89.346444
1,2005-09-30,NaN,NaN,NaN,NaN,NaN,NaN,96.534935,NaN,NaN,...,15.871030,62.047600,62.047600,51.564443,93.419152,93.419152,93.419152,89.205829,89.205829,89.205829
2,2005-10-31,NaN,NaN,NaN,NaN,NaN,NaN,96.471981,NaN,NaN,...,16.097034,61.961402,61.961402,52.128179,93.798649,93.798649,93.798649,89.420052,89.420052,89.420052
3,2005-11-30,NaN,NaN,NaN,NaN,NaN,NaN,96.472460,97.503718,97.503718,...,16.909461,62.512187,62.512187,52.846167,93.905481,93.905481,93.905481,89.080571,89.080571,89.080571
4,2005-12-31,NaN,NaN,NaN,NaN,NaN,NaN,97.655170,97.996633,97.996633,...,20.332567,62.126714,62.126714,60.209553,91.484261,91.484261,91.484261,93.541597,93.541597,93.541597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,2025-06-30,99.000611,99.000611,99.000611,97.892807,97.892807,97.892807,99.504106,99.999989,99.999989,...,77.992514,97.671326,97.671326,93.137308,97.224061,97.224061,97.224061,99.363345,99.363345,99.363345
239,2025-07-31,99.073587,99.073587,99.073587,97.926685,97.926685,97.926685,99.627776,100.000012,100.000012,...,78.723921,97.738047,97.738047,93.201061,97.311733,97.311733,97.311733,99.490787,99.490787,99.490787
240,2025-08-31,99.090801,99.090801,99.090801,97.910641,97.910641,97.910641,99.625314,100.000009,100.000009,...,78.425465,97.698057,97.698057,93.258100,97.260548,97.260548,97.260548,99.488512,99.488512,99.488512
241,2025-09-30,98.275030,98.275030,98.275030,97.371288,97.371288,97.371288,98.765681,99.999991,99.999991,...,77.311625,96.750014,96.750014,92.399361,97.000819,97.000819,97.000819,98.463638,98.463638,98.463638


In [ ]:
# 欠損確認（Weight (%)）
dfs_weight_sum = []
with sqlite3.connect(financials_db_path) as conn:
    for variable in table_names:
        df_factor = pd.read_sql(
            f"SELECT `date`, `P_SYMBOL`, `value` FROM `{variable}`",
            con=conn,
            parse_dates=["date"],
        )
        merged_df = (
            pd.merge(df_weight, df_factor, on=["date", "P_SYMBOL"], how="outer")
            .rename(columns={"value": variable})
            .dropna(subset=["Weight (%)", variable], how="any", axis=0)
        ).fillna(np.nan)

        g = (
            pd.DataFrame(merged_df.groupby(["date"])["Weight (%)"].agg("sum"))
            .reset_index()
            .assign(variable=variable)
        )

        dfs_weight_sum.append(g)

df_weight_sum = pd.concat(dfs_weight_sum, ignore_index=True)
df_weight_sum = pd.pivot(
    df_weight_sum, index=["date"], columns="variable", values="Weight (%)"
).reset_index()
display(df_weight_sum)

KeyboardInterrupt: 